In [98]:
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import pandas as pd
import time

In [99]:
# ### you can use this code to extract the text from news article link

# def extract_text(url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')

#     for script in soup(["script", "style"]): 
#         script.extract()

#     text = soup.get_text()

#     # break into lines and remove leading and trailing space on each
#     lines = (line.strip() for line in text.splitlines())

#     # break multi-headlines into a line each
#     chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

#     # drop blank lines
#     text = '\n'.join(chunk for chunk in chunks if chunk)

#     return text

# url = 'url' # Replace with your URL
# text = extract_text(url)
# print(text)


In [100]:
## For showing a sample calculation of Bias, 
##I have uploaded the dataset (11 News articles) regarding Indian prime minister Narendra Modi's 2023 US visit

In [101]:
url2 = 'https://www.indiatoday.in/india/story/modis-trip-was-most-extraordinary-in-history-of-us-india-relations-garcetti-2409586-2023-07-21' 


sentence2 = """

Modi's trip was most extraordinary in history of US-India relations: Garcetti - India Today India
 GarcettiModi's trip was most extraordinary in history of US-India relations: GarcettiWe've had some very large moments in the past... But this was probably the deepest and widest agenda of not just aspirations, but real deliverable things for our economy, for defence, said America's Ambassador to India Eric Garcetti. Prime Minister Modi last month travelled to the US on his maiden state visit during which he held wide-ranging talks with President Joe Biden. (Reuters photo)By Press Trust of India: Prime Minister Narendra Modi's historic state visit to the United States last month was the "most extraordinary" visit in the history of this bilateral relationship, America's Ambassador to India Eric Garcetti said on Thursday."We've had some very large moments in the past... But this was probably the deepest and widest agenda of not just aspirations, but real deliverable things for our economy, for defence, the four Ps as I call them, peace, prosperity, 
 the planet and the people that we represent," he said.advertisementPrime Minister Modi last month travelled to the US on his maiden state visit during which he held wide-ranging talks with President Joe Biden. Modi also addressed the joint session of the US Congress, becoming the first Indian leader to do so twice."It was the most extraordinary visit, I think, certainly in the history of US-India relations,” Garcetti told PTI in an interview.Since the historic state visit, he noted, there has been no let up. "Both my colleagues in India have said that the Indian government has been following up with a series of meetings to implement the agreements, and we (are) here. One of the reasons I'm in Washington is to keep that momentum going as our president comes to Delhi for the G-20 meeting,” he said.There are so many key deliverables from the visit. Obviously one that got a lot of attention is the GE jet engine deal, which will fundamentally change the level of cooperation we have on the next generation of Indian fighter jets and also integrate our militaries and our defence together, he noted."But I can also think of other things like the relaxing of the tariffs and the trade war that we had during the Trump years in which steel was taxed here, and we had American agricultural program products taxed in India. That's a huge change, making something for Indian workers that are in the steel sector and for our agricultural products finally to have free flow. Educational exchange we grew and expanded,” he said."The last area is our critical emerging technologies: whether it's artificial intelligence, whether it's quantum computing, whether it's space, or whether it's the telecommunications, I think India and the US really want to cement a deep friendship and co-production, the research and development together of the future underscored by our values,” he said.READ | US-India friendship among most consequential in the world: President Biden"We can either use technology to hurt people, divide people, or we can use it to connect people, protect people, even detect diseases and heal people. So to me, that was one of the best outcomes, not just for India and the United States, but also for the world,” Garcetti said.advertisementObserving that sky is the limit for US India relationship, he said since the two sides are collaborating in space, maybe even the sky's not the limit. "We are going from the depths of the ocean to the heavens, in terms of the work that we're doing. Space, the maritime domain," he said."I think by the end of the year, our leaders will have seen each other face to face six or seven times in a single year. We know the rapport that they have together, but it also reflects the hard work and integration of our governments. That in turn reflects business investment and the people-to-people ties. We have a record number of visas that we're going to process this year in our consulates and in our embassy from India. So that's reflecting, I think that India loves America and America loves India," he said.The former mayor of Los Angeles has been in India for the last few months, during which he says he has fallen in love with India all over again.advertisementREAD | Jaishankar explains how PM Modi's US visit was different from other PMs' visits"The people, the culture, the food, the diversity, the struggle, the success. This is in so many ways, the future, but it also links us in so many ways to the past. It's a unique country, a crossroads and a bridge. And for me, I want to introduce more Americans to India. Part of my job as an ambassador is not just to bring India to America, but to bring America to India so that we can see that,” he said."I do think that this chapter, I'm noticing it whether it's in the buildings coming up in Hyderabad, whether it's the infrastructure that I saw in Mumbai and Kolkata that's being done, or just even at the village level chaiwala using UPI to accept a payment that India isn't just someone's back office. It isn't just coming out of poverty. It's producing ideas, it's producing companies, it's producing things that are improving people's lives and potentially throughout the world,” he said."So I think if we strengthen this bridge (in) both directions, really, we can do anything together, whether it's health, whether it's defence, whether it's economic prosperity or even women's empowerment,” Garcetti added. 

"""

In [102]:
# ##### GPT4 prompt output for teh above article


# Based on the provided text, here is a breakdown of positive and negative points related to Prime Minister Narendra Modi's trip to the US:

# **Positive Points**:
# 1. Eric Garcetti, America's Ambassador to India, described Prime Minister Narendra Modi's state visit to the US as the "most extraordinary" visit in the history of US-India bilateral relations.
# 2. The agenda of this visit was described as the deepest and widest, encompassing not just aspirations but also real deliverable things.
# 3. The topics of the agenda included the economy, defense, peace, prosperity, the planet, and the people represented by both countries.
# 4. Modi addressed the joint session of the US Congress, a significant achievement as he became the first Indian leader to do so twice.
# 5. There has been continued momentum since the historic state visit, with multiple follow-up meetings to implement agreements.
# 6. A GE jet engine deal that will enhance the cooperation on next-generation Indian fighter jets and integrate militaries and defense of both countries.
# 7. Relaxation of tariffs and an end to the trade war that had been initiated during the Trump years.
# 8. Expansion of the educational exchange program.
# 9. Collaboration in critical emerging technologies like artificial intelligence, quantum computing, space, and telecommunications, focusing on co-production, research, and development.
# 10. Garcetti emphasized the potential positive impact of technology not only for both countries but also for the world.
# 11. Increasing collaboration in space and maritime domains.
# 12. Multiple face-to-face meetings between leaders reflect the strong ties and integration of both governments.
# 13. A record number of visas expected to be processed, reflecting strong people-to-people ties.
# 14. Garcetti expressed his admiration for India's culture, diversity, and progress.
# 15. Mention of India's progress in various sectors, from infrastructure in major cities to technology acceptance at the grassroots level.
# 16. Garcetti's emphasis on the potential of strengthening the bridge between India and the US in various domains, including health, defense, economic prosperity, and women's empowerment.

# **Negative Points**:
# - No explicit negative points were mentioned in the provided text regarding Prime Minister Narendra Modi's trip to the US.

# Overall, the provided text is overwhelmingly positive in its assessment and coverage of Prime Minister Narendra Modi's trip to the US.

In [103]:
num_pos_pts2 = 16
num_neg_pts2 = 0

In [104]:
url3 = 'https://indianexpress.com/article/india/narendra-modi-us-state-visit-joe-biden-india-8683327/' 


sentence3 = """

PM Modi wraps up US visit: Together, shaping dreams | The Indian Express
    
PM Modi wraps up US visit: Together, shaping dreams
“The partnership between India and the United States will make the world better in the 21st Century. You all play a crucial role in this partnership,” said PM Modi while addressing the Indian diaspora at Ronald Reagan Center, Washington.

Narendra Modi addresses Indian diaspora in the US. (Photo: PMO)TOGETHER INDIA and the US are not just forming policies and agreements, “we are shaping lives, dreams and destinies”, Prime Minister Narendra Modi said Saturday, concluding his three-day state visit to the US. He said the partnership between India and the US will make the world better in the 21st century.
In his farewell speech to the Indian-American community, the Prime Minister said, “In the last three days, a new and glorious journey has started for India-US relations.” He said the world is watching the two great democracies strengthen their bond.
“The partnership between India and the United States will make the world better in the 21st century. You all play a crucial role in this partnership,” Modi said, addressing a gathering of the Indian diaspora at Ronald Reagan Center in Washington. The diaspora will play a big role in helping realise the true potential of the two countries’ ties and this is the right time to invest more and more in India, he said.
He cited massive infrastructure projects, which are in progress in the country, to assert that India’s new growth story is being written in many tier-2 and tier-3 cities. India is the mother of democracy and the US is the champion of modern democracy, the Prime Minister said, also lauding President Joe Biden as a seasoned and sorted leader, and praising his efforts in taking the India-US ties to a new high. “I publicly praise his efforts,” he said.
India is growing rapidly and this is transforming several lives. 
Earlier in the day, addressing business leaders and philanthropists from India and the US at the Kennedy Centre, Modi said, “The biggest driving force of India’s growth story is the aspiration of the Indian people, which is not very different from the American dream.” He said “extreme poverty is fast diminishing in India, while the middle-class and neo middle class are rapidly expanding”.
“Whatever India is going to do in order to fulfil the aspirations of these people, opens up immense possibilities for America as well,” he said, mentioning an increasing demand in India in the aviation sector. “To fulfil this demand, Indian airlines are ordering hundreds of aircraft from American companies, and jobs are being created here.”
The India-USA partnership is driven by conviction and compassion. It is also a friendship that has bipartisan support.
Modi said history bears witness to the fact that a strong India augurs well for the world. Citing the pandemic era, he said, “When the world needed medicines, India increased its production and sent medical supplies to more than 150 countries. India also sent vaccines to more than 100 countries.”
At the diaspora event later, the Prime Minister said, “India is now witnessing an unprecedented digital revolution. The changing face of India will surprise you. ” He said the biggest driver for India’s progress today is its people’s self-confidence. “Hundreds of years of slavery had taken away this self-confidence from us,” he said.
Delighted to interact with the diaspora at the community programme in Washington DC.
Modi also mentioned a slew of measures and takeaways that were discussed and agreed upon during his visit to the US. “General Electric’s decision to manufacture fighter plane engines in India will be a milestone in India’s defence sector,” he said. “This new journey is of the cooperation of Make in India – Make for the world,” he said, referring to agreements on technology transfer, boosting manufacturing and strengthening industrial supply chain.
Here's how various American dailies and leading news outlets covered the Indian PM's visit, a day after his key engagements.
“It has been decided that now, renewal of H1B Visa will not require one to move out of America. A pilot project in this regard will be started soon this year. This will help thousands of IT sector employees here,” said the Prime Minister, adding that this facility may also be extended to L-category visa (intra-company transfer visa) depending on experience. India will open a new consulate in Seattle and two in other cities, while the US is opening consulates in Ahmedabad and Bengaluru.

"""

In [105]:
# Based on the provided text, here's a breakdown of the positive and negative points related to Prime Minister Narendra Modi's trip to the US:

# **Positive Points**:
# 1. PM Modi emphasizes that the partnership between India and the US is shaping lives, dreams, and destinies.
# 2. The partnership is believed to improve the world in the 21st century.
# 3. The Indian diaspora plays a vital role in realizing the potential of India-US ties.
# 4. India is witnessing growth in many tier-2 and tier-3 cities with massive infrastructure projects.
# 5. PM Modi lauds President Joe Biden for elevating India-US ties.
# 6. The aspiration of the Indian people drives India’s growth.
# 7. Poverty in India is rapidly diminishing, and the middle class is growing.
# 8. Indian airlines are placing orders with American companies, leading to job creation in the US.
# 9. India provided medicines and vaccines to many countries during the pandemic.
# 10. India is undergoing a digital revolution.
# 11. The decision by General Electric to manufacture fighter plane engines in India is seen as a milestone.
# 12. Cooperation on technology transfer, manufacturing, and strengthening the industrial supply chain has been emphasized.
# 13. A new system regarding the renewal of H1B Visas will benefit many IT sector employees in the US.
# 14. New consulates will be opened by both countries, enhancing diplomatic relations.

# **Negative Points**:
# 1. PM Modi referenced that hundreds of years of slavery had taken away India's self-confidence.

# Based on the provided text, there are 14 positive points and 1 negative point related to PM Narendra Modi's trip to the US.

In [106]:
num_pos_pts3 = 14
num_neg_pts3 = 0

In [107]:
url4 = 'https://www.thehindu.com/news/national/india-us-partnership-is-of-conviction-shared-commitments-compassion-pm-modi/article67003512.ece' 


sentence4 = """
‘This is the moment’: PM Modi tells diaspora and business leaders as U.S. visit concludes - The Hindu
PM Modi also noted that in the last two and two and a half years, U.S. companies have invested more than $16 billion in India
  President Joe Biden and Prime Minister Narendra Modi meet with senior officials and CEOs of American and Indian companies in the East Room of the White House in Washington, on June 23, 2023.

Prime Minister Narendra Modi wrapped up his two-and-a-half day visit to Washington DC with diaspora events, including a speech urging the business community to invest in India, promoting India’s economic prospects.Speaking at the Kennedy Centre for the Performing Arts, Mr Modi urged an audience largely comprised of members of the Indian diaspora to make use of the momentum in the bilateral relationship, as he praised them.

When India grows, the world grows, says PM Modi in address to joint session of U.S. Congress
“You have lived the American dream,” he said, at an event organized by the U.S. India Strategic Partnership Forum.“You have shown how these dreams can be fulfilled with a strong result,” he added.With India and the U.S. announcing a raft of agreements across sectors such as defence, health, climate and emerging tech, Mr Modi promoted the relationship and the gains made in the last two days. The bilateral relationship was not just a partnership of convenience but a “partnership of conviction”, Mr Modi said, as he highlighted the bipartisan support it enjoys in the U.S. Congress.Also read: Coming together of Indian talent and U.S. technology guarantees brighter future: PM Modi at meet of top CEOsHe went through a list of U.S. states that were benefitting from defence deals with India, and promoted the Indian economy, its youthful population and compared the aspirations of Indians to the American dream.“Today, the share of private consumption and India’s GDP is the highest it has been in 1.5 decades,” he said, adding that extreme poverty is “ending rapidly” in India and the middle class is growing.He urged the business community to make use of the opportunities this presented.

PM Modi’s U.S. visit yields many vital agreements
“This partnership between India and America is in the interest of both countries, is in the interest of the people of both countries, and that is why it is so important to strengthen it,” he said.“This is the moment,” he said, adding that the governments of the two countries had done the groundwork and it was up the business community to make use of the opportunities now.Mr Modi was joined on stage by U.S. Secretary of State Antony Blinken, USISPF Chairman John Chambers and USISPF CEO Mukesh Aghi. External Affairs Minister S Jaishankar and India’s U.S. Ambassador Taranjit Singh Sandhu were among those in the audience.Following the Kennedy Center event, Mr Modi attended a diaspora event at the Reagen Center.“Concluding a very special USA visit,” Mr Modi tweeted. “Our nations will keep working together to make our planet a better place for the coming generations,” he said, posting a campaign-style video of his trip, shortly before emplaning for Cairo, for another state visit  


"""

In [108]:
# Based on the provided text from "The Hindu", here's a breakdown of the positive and negative points related to Prime Minister Narendra Modi's trip to the US:

# **Positive Points**:
# 1. U.S. companies have invested more than $16 billion in India in the last 2 to 2.5 years.
# 2. PM Modi's emphasis on urging the business community to invest in India.
# 3. Praising the Indian diaspora for achieving the American dream and producing strong results.
# 4. A variety of agreements have been announced between India and the U.S. across several sectors including defense, health, climate, and emerging tech.
# 5. Emphasis on the bilateral relationship being a "partnership of conviction" and its bipartisan support in the U.S. Congress.
# 6. Highlighting U.S. states benefiting from defense deals with India.
# 7. Promotion of the Indian economy, its young population, and the similarity of the aspirations of Indians to the American dream.
# 8. The high share of private consumption in India’s GDP and its significance in the past 1.5 decades.
# 9. Pointing out the rapid decline of extreme poverty in India and the growth of the middle class.
# 10. PM Modi stressing the mutual benefits of the India-U.S. partnership.
# 11. Emphasis on the preparedness by both governments and the opportunities available for the business community.
# 12. Joining of prominent figures on stage, such as the U.S. Secretary of State Antony Blinken.
# 13. The concluding sentiment of both nations working together for future generations.

# **Negative Points**:
# None explicitly mentioned in the text.

# Based on the provided text, there are 13 positive points and no negative points related to PM Narendra Modi's trip to the US.

In [109]:
num_pos_pts4 = 13
num_neg_pts4 =0

In [110]:
url5 = 'https://www.firstpost.com/explainers/pm-narendra-modi-us-visit-defence-jet-engine-deal-space-technology-joe-biden-12783732.html' 


sentence5 = """

Why PM Modi’s US visit is a big hit
PM Narendra Modi has concluded his three-day US visit, which saw a flurry of mega announcements. From fighter jet engines deal to cooperation in space exploration, the trip has boosted the ties between the two nations
FP Explainers
PM Narendra Modi's US visit saw a flurry of deals and announcements between the two countries. Reuters
Prime Minister Narendra Modi has left for Egypt after wrapping up his first state visit to the United States, which saw a series of mega deals and announcements. Besides a bilateral talk with US president Joe Biden, an address to a joint session of the US Congress, and attending a grand state dinner, the Indian leader also met top tech and business leaders during his three-day state visit to Washington.
In a joint statement issued by India and the US, the two nations asserted they are “among the closest partners in the world – a partnership of democracies looking into the 21st Century with hope, ambition, and confidence”. Let’s take a look at why Prime Minister Modi’s US visit was so fruitful.
Defence deals
French President Macron welcomes PM Modi in Paris, tweets in HindiWATCH: PM Modi meets Crown Prince Mohammed bin Zayed Al Nahyan in UAEIndia and the US entered key agreements in the defence sector as Modi visited the US.
One of the most significant developments included American company General Electric announcing a collaboration with India’s state-owned Hindustan Aeronautics Ltd to produce fighter jet engines for the Indian Air Force (IAF). The deal will lead to a “significant amount of transfer of technology and work being done indigenously”.
The joint statement by Biden and Modi referred to the jet engine deal as a “landmark” and “trailblazing initiative”.
New Delhi will also procure 31 High Altitude Long Endurance (HALE) drones, MQ-9B, from General Atomics which intends to boost the intelligence, surveillance, and reconnaissance capabilities of the Indian armed forces.
PM Narendra Modi with US president Joe Biden, Apple CEO Apple and Microsoft’s Satya Nadella at the White House. AP
The US Navy has also finalised a Master Ship Repair Agreement (MSRA) with Larsen and Toubro Shipyard in Kattupalli (Chennai) and is in talks with Mazagon Dock Limited in Mumbai and Goa Shipyard (Goa). This will allow US Navy ships in the region to undergo service and repair at Indian shipyards.
India and the US have also progressed to operationalise tools for increasing defence cooperation as well as strengthening undersea domain awareness. In a first, the two countries have agreed to keep three Indian liaison officers in US command, noted Indian Express.
Biden and Modi also hailed the launch of the US-India Defense Acceleration Ecosystem (INDUS-X) – a network of universities, startups, industry and think tanks. This will facilitate a “joint defence technology innovation and co-production of advanced defence technology between the respective industries of the two countries”, reported ThePrint.
Space sector
India has signed the Artemis Accords, a US-led alliance aimed at guiding the safe exploration of the Moon and, eventually, Mars. Since its inception in 2020, as many as 25 other countries are a part of this international collaboration.
US’ National Aeronautics and Space Administration (NASA) and Indian Space Research Organisation (ISRO) will also develop a strategic framework for human spaceflight cooperation by the end of this year.
Moreover, ISRO astronauts will receive advanced training from NASA in order to kickstart a joint effort to the International Space Station in 2024.
Visas
The US government will roll out a pilot this year to adjudicate visa renewals in America for some petition-based temporary work visas, including for Indian nationals. This could eventually expand to cover H1B and L visa holders and other categories by 2024, benefitting professionals as they will not have to travel overseas for the renewal of their work visas.
ALSO READ: ‘We’re shaping lives, dreams, destinies’: Why PM Modi’s diaspora speech in Washington is significant
Semiconductor supply chains
American technology firm Micron Technology announced its plan to invest up to USD 825 million to build a new $2.75 billion semiconductor assembly and test facility in Gujarat with support from the Indian government, reported The Hindu.
Applied Materials will set up a Semiconductor Centre for Commercialization and Innovation in India to boost semiconductor supply chain diversification.
In the joint statement, Biden and Modi also appreciated Lam Research’s proposal to train 60,000 Indian engineers through its Semiverse Solution virtual fabrication platform.
Critical minerals partnership
India has joined Minerals Security Partnership (MSP), a US-led initiative for diverse and sustainable critical energy mineral supply chains. Launched in 2022, the MSP has 12 other partner countries, in addition to the European Union.
India’s Epsilon Carbon Limited has decided to invest $650 million in a greenfield electric vehicle battery component factory, which, as per Indian Express, will be the “largest ever Indian investment in the US electric vehicle battery industry.”
PM Narendra Modi addressed a joint meeting of Congress at the Capitol in Washington. AP
New consulates
To enhance people-to-people ties and travel, both sides will open new consulates in each other’s countries.
While the US intends to open consulates in Bengaluru and Ahmedabad, India is planning to open a consulate in Seattle later in 2023.
Currently, there are five US consulates in India. India also aims to announce two more consulates in the US, as per the joint statement.
Other key announcements
The two countries have established a joint Indo-US Quantum Coordination Mechanism “to facilitate collaboration among industry, academia, and government, and our work toward a comprehensive Quantum Information Science and Technology agreement”, said the joint statement.
The US will join the Indo-Pacific Oceans Initiative, which was inaugurated by Modi in 2015 to ensure a safe, secure, and stable maritime domain, as per Indian Express.
Modi and Biden also launched two Joint Task Forces on advanced telecommunications, with a focus on Open RAN and research and development in 5G and 6G technologies.
Google CEO Sundar Pichai informed Modi on Friday (23 June) about his plans to invest $10 billion in India’s digitisation fund.
A $2 million grant programme will be set up under the US-India Science and Technology Endowment fund for the joint development and commercialisation of Artificial Intelligence (AI) and quantum technologies.
The US-India Commercial Dialogue will launch a new “Innovation Handshake” to connect the startup ecosystems of the two countries.
India’s Sterlite Technologies Limited has invested $100 million to build an optical fibre cable manufacturing unit near Columbia, South Carolina, which will lead to $150 million in annual exports of optical fibre from India.
India and the US have decided to continue to cooperate to meet their national climate and energy goals as well as stand together to counter global terrorism.
The US National Cancer Institute will encourage the US and Indian scientists to join hands to develop AI-enabled digital pathology platform.
As per Indian Express, the US National Institute of Diabetes and Digestive and Kidney Diseases will sign a deal with the Indian Council of Medical Research to “further basic, clinical, and translational research on diabetes

"""

In [111]:
# Based on the provided text titled "Why PM Modi’s US visit is a big hit," here's a breakdown of the positive and negative points related to Prime Minister Narendra Modi's trip to the US:

# **Positive Points**:
# 1. A series of mega deals and announcements between the two countries.
# 2. A joint statement emphasizing the strong partnership between the two democracies.
# 3. Defense sector agreements, including fighter jet engine production.
# 4. “Significant amount of transfer of technology and work being done indigenously” from the defense deal.
# 5. The jet engine deal was labeled as a “landmark” and “trailblazing initiative”.
# 6. Procurement of 31 High Altitude Long Endurance (HALE) drones by India.
# 7. Master Ship Repair Agreement (MSRA) between the US Navy and Indian shipyards.
# 8. Progress in operational tools for defense cooperation and undersea domain awareness.
# 9. Launch of the US-India Defense Acceleration Ecosystem (INDUS-X).
# 10. India signing the Artemis Accords for lunar and Martian exploration.
# 11. Collaboration between NASA and ISRO for human spaceflight.
# 12. Training of ISRO astronauts by NASA for a joint mission to the International Space Station.
# 13. US introducing visa renewal procedures benefitting professionals.
# 14. Investments and initiatives in the semiconductor industry by American companies in India.
# 15. India joining the Minerals Security Partnership (MSP).
# 16. Indian company investing in the US electric vehicle battery component industry.
# 17. Opening of new consulates to enhance bilateral ties.
# 18. Establishment of a joint Indo-US Quantum Coordination Mechanism.
# 19. US joining the Indo-Pacific Oceans Initiative.
# 20. Launch of Joint Task Forces focusing on telecommunications tech.
# 21. Google's $10 billion investment in India’s digitisation fund.
# 22. Introduction of a $2 million grant program for AI and quantum tech.
# 23. Launch of “Innovation Handshake” to connect the startup ecosystems.
# 24. Sterlite Technologies Limited's investment for an optical fibre cable unit in the US.
# 25. Commitment to cooperate on national climate and energy goals and counter-terrorism.
# 26. Collaborative research on AI-enabled digital pathology between US and Indian scientists.
# 27. Research collaboration on diabetes between the US and India.

# **Negative Points**:
# None explicitly mentioned in the text.

# Based on the provided text, there are 27 positive points and no negative points related to PM Narendra Modi's trip to the US.

In [112]:
num_pos_pts5 = 27
num_neg_pts5 = 0

In [113]:
url6 = 'https://www.ndtv.com/india-news/us-media-covers-pm-narendra-modis-state-visit-with-focus-on-tech-defence-deals-4147704' 

sentence6 = """

US Media Covers PM Modi's State Visit With Focus On Tech, Defence DealsSearchNewsIndia NewsUS Media Covers PM Modi's State Visit With Focus On Tech, Defence Deals US Media Covers PM Modi's State Visit With Focus On Tech, Defence Deals
The style section of The Washington Post has a half-page photograph of PM Modi flanked by the Bidens in the White House for the lavish state dinner. India NewsPress Trust of IndiaUpdated: June 24, 2023 1:42 am ISTPM Modi is currently in Washington on a state visit at the invitation of President Joe Biden. (File)Washington: Leading American newspapers highlighted Prime Minister Narendra Modi's first state visit to the US, with prominent coverage including reports on technology and defence agreements signed between the two countries.In a full-page coverage inside The Washington Post, the paper notes 'In Prime Minister's address before Congress, Modi puts his focus on growth.' "Lawmakers welcomed Modi to the House Chamber warmly, giving him a loud standing ovation. Some in the gallery chanted his name as lawmakers lined up to shake his hand as he made his way to the dais," the report said.Front-page coverage of PM @narendramodi's visit to USA in major US dailies.Truly a transformational visit! pic.twitter.com/yjdkAiwmFq— Arindam Bagchi (@MEAIndia) June 23, 2023Prime Minister Modi is currently in Washington on a state visit at the invitation of President Joe Biden. On Thursday, both leaders held a historic summit during which the two countries signed an agreement to jointly produce jet engines in India to power military aircraft and a deal on the sale of armed drones to India.Another article "US, India announce agreements on technology, defence" said, "Biden and Modi announced several agreements, including a deal to jointly produce a GE fighter jet engine in India and efforts to shore up supply chains for microchips and other key technologies."The style section of The Washington Post has a half-page photograph of Prime Minister Modi flanked by the Bidens in the White House for the lavish state dinner.The article accompanying the report says "All the pomp but light on the glee."A photograph of Prime Minister Narendra Modi, greeting US lawmakers with a 'Namaste' during his address at the US Congress, is on the front page of The New York Times, with the caption "At a joint session of Congress on Thursday, Prime Minister Narendra Modi of India avoided using the words "Russia" and "China."A full page of coverage of the visit is splashed inside the paper, with a photograph of Biden, with his hand on PM Modi's shoulder, with the Washington Monument in the foreground, the two leaders watching the ceremonial guard of honour and the thousands of Indian-Americans who had gathered at the South Lawns of the White House on a rainy Thursday morning.The detailed NYT report notes that "to mark their ties, the two leaders rolled out a long list of joint initiatives on telecommunications, semiconductors, artificial intelligence and other areas."The Financial Times carried a photograph of the two leaders standing for the national anthems on the South Lawn of the White House during the ceremonial welcome, with the caption 'Stronger Ties Biden and Modi commit to a 'defining' relationship between democracies'.A detailed report on the technology and defence agreements signed between the two leaders is on the second page."Joe Biden welcomed Narendra Modi to the White House yesterday for a state visit during which the US and Indian leaders will announce defence and technology deals including a purchase of US spy drones."The accords, which include efforts to boost space cooperation, mark a big push by the US to draw New Delhi into its orbit as part of a plan to work with allies and partners to counter China",


"""

In [114]:
# Based on the provided text titled "US Media Covers PM Modi's State Visit With Focus On Tech, Defence Deals," here's a breakdown of the positive and negative points related to Prime Minister Narendra Modi's trip to the US:

# **Positive Points**:
# 1. Leading American newspapers highlighted PM Narendra Modi's state visit prominently.
# 2. The Washington Post covered the Prime Minister's address, noting his focus on growth.
# 3. Lawmakers in the US gave Modi a warm reception, with a standing ovation and chants of his name.
# 4. PM Modi's visit is described as "transformational" in a tweet.
# 5. A historic summit was held between PM Modi and President Joe Biden, resulting in significant agreements.
# 6. Joint production of jet engines in India was agreed upon.
# 7. A deal on the sale of armed drones to India was signed.
# 8. Agreements on technology and defense were announced.
# 9. The New York Times featured a photograph of PM Modi on its front page, highlighting his greeting to the US lawmakers.
# 10. Coverage in the New York Times mentioned joint initiatives on telecommunications, semiconductors, and artificial intelligence.
# 11. The Financial Times noted the commitment of both leaders to a 'defining' relationship between democracies.
# 12. The visit is seen as part of a strategy by the US to collaborate with allies and partners, including India, to balance against China's influence.

# **Negative Points**:
# 1. The article accompanying the photograph in The Washington Post had a comment, "All the pomp but light on the glee," which could be interpreted as suggesting that while the event was grand, it lacked genuine enthusiasm or warmth.
# 2. PM Modi's avoidance of using the words "Russia" and "China" during his address could be perceived as him sidestepping significant geopolitical topics or being diplomatically cautious, which might be seen as a missed opportunity to make a clear stance.

# Based on the provided text, there are 12 positive points and 2 negative points related to PM Narendra Modi's trip to the US as covered by the US media.

In [115]:
num_pos_pts6 = 12
num_neg_pts6 = 2

In [116]:
url1 = 'https://thewire.in/security/modi-us-visit-security-vulnerability' 


sentence1 = """
Modi’s US Visit Has Increased India’s Vulnerabilities About UsSupport UsLoginBecome A SupporterHindiMarathiUrduHomecategoryopinionVideoTermsPrivacyAbout Us©The Wire | 2018PoliticsEconomyWorldSecurityLawScienceSocietyCultureSecurityModi’s US Visit Has Increased India’s VulnerabilitiesAs strategic autonomy makes way for a tighter military embrace with Washington, India will find that its room for manoeuvre in the face of emerging challenges has narrowed.Prime Minister Narendra Modi and US President Joe Biden at the White House, in Washington, DC on June 22, 2023. Photo: PIBPravin SawhneyDiplomacySecuritySouth AsiaWorld26/Jun/2023Geopolitical rivalry with China has brought India and the US into the tight strategic embrace which was on ample display during the recent state visit to Washington of Prime Minister Narendra Modi. The US needs India for global geopolitics while India wants the US for regional geopolitics.
The only problem is that the US, having identified China as its sole geopolitical competitor in this century – with the capability to match it in economy, technology, diplomacy, and military – has failed to accept that its deterrence (military power) model of the Cold War is unsuitable against the Chinese geopolitical model, which is based more on global cooperation for prosperity than military power alone.
Even in a fragmented world, the Chinese model cannot fail. Changing the game from free trade to weaponised trade by ‘decoupling’ or ‘de-risking’ value chains and supply chain networks from dependence on China – still the world’s biggest trading partner – will only add to geopolitical tensions with the global economy heading towards recession.
Unmindful of the disastrous fall-out of the Ukraine war on the global economic order led by the US dollar, the reluctance of its regional allies to openly confront China and the own recent experience of its top diplomat being lectured by Chinese supremo Xi Jinping, the US has decided to bet on India to build it as a military bulwark against China in the Indian Ocean Region (IOR).
The long-term bet is based on the assumption that Prime Minister Modi will win the 2024 general elections and will not normalise India’s relations with China in this decade — the time it would take for deliverables promised by each side to take shape. The other assumption is that, if push comes to shove, India, as claimed by its political and military leadership, will be able to take on the People’s Liberation Army (PLA) in a border war.
Ironically, the huge jubilation in India regarding the Modi visit missed the point that under the new transactional arrangement, India would end up giving far more, including its strategic autonomy, than it will get in return – which will be further accentuated by the US’s irrepressible urge (as seen during the 2005-2008 civil nuclear deal negotiations) to shift the goalposts.
The US agenda
So, what does the US want from India?
It wants commonality of military equipment by slowly weaning India (the largest arms importer in the world) away from Russia, its traditional, affordable, and trusted partner. It wants to improve Indian naval dockyards to serve as temporary military bases for its assets (vessels of all hue, including nuclear submarines and carriers) in the IOR. It also wants the Indian military (especially the navy) to do advanced exercises bilaterally and multi-laterally with the Quad navies for developing interoperability for combat in the IOR, which India considers its backwaters. Since the character of war has changed with new-age technologies, the US wants to pull up the technological level of the Indian military through the newly crafted India-United States Defence Acceleration Ecosystem (INDUS-X), which is part of the bigger Innovation on Critical and Emerging Technologies (ICET) framework whereby it will be able to operate within the US Indo-Pacific Command (INDOPACOM) defence network under its ‘integrated deterrence’ strategy by the end of this decade.
All this has become possible as India (which is neither a military ally nor a non-NATO ally) has signed the US military’s four foundational agreements which qualify it to be part of the US’s new age networks. Thus, by 2030, India will fulfil all the requirements for interoperability: commonality of equipment, advanced military exercises, combat support facilities like MROs (maintenance, repair, operations) from the US, and familiarity with various war contingencies in the IOR.
Moreover, as part of iCET, India will adopt US rules, regulatory, norms, and standards in the new age fourth industrial technologies for trade and commerce since US companies will have first user advantage in India. Thus, given the certainty of the splinternet (bifurcation of the Internet and the value supply chains) based on US and Chinese technologies, India will get strategically isolated from its South Asian neighbours, all of which are onboard the BRI (Belt and Road Initiative), which will be based on the Chinese internet by 2030.
GE engines and technology transfer
Let’s now examine the deals individually, starting with General Electric’s memorandum of understanding with Hindustan Aeronautics Limited (HAL).
According to the GE press release, it will deliver 99 F-414 engines for Indian Air Force (IAF) Light Combat Aircraft (LCA) Mk2 programme. GE will also help with the ‘prototype development, testing, and certification of the AMCA (Advanced Medium Combat Aircraft) programme with its F-414-INS6 engine’. Moreover, GE will provide 99 F-404 engines for the LCA Mk1 programme. Both GE engines (F-404 and F-414) have already been part of the development of LCA Mk1 and LCA Mk2 programmes.
Three observations are in order. One, there will be no transfer of technology (ToT) of turbofan and casting technologies and even metallurgy formulae which make up almost 95 per cent of the engine’s Intellectual Property Rights (IPRs). It stands to logic that GE, which has spent huge amounts of time, resources, and talent creating the engine, cannot hand over its IPRs to India and create a competitor for itself. All India will get is engine assembly rights (called indigenous production in India), like we have for Russian AL-31FP engines for the Su-30MKI, which are being made in Koraput since 2004. The remaining five per cent ToT which includes tools for engine maintenance may be allowed by GE after the US Congress clears it. Thus, the IAF will be saddled with the burden of maintaining yet another engine besides British, Russian, and French.
Two, the US will make a strong bid for the IAF’s urgent operational need of 114 fighter jets under the Multi-Role Fighter Aircraft (MRFA) and the Indian Navy’s carrier-borne fighter programmes, in both of which Boeing’s Super Hornet, powered by GE-F414 engines, is a contender. Cheaper than the Rafale, the Super Hornet has been in US military service since 1999 with various upgrades. The IAF will be saddled with one more fighter, when it wants to reduce its types of combat aircraft. The Super Hornet, if it comes to the IAF and the Indian Navy, will be Prime Minister Modi’s choice rather than the Services’ Headquarters, which favours Rafale.
Three, while care will be taken by the Indian side to make an impregnable legal case against US sanctions on GE F-414, the US is not known to care much for legalities. A case in point is the freedom of navigation patrols by US Navy vessels in the South China Sea (SCS) and Taiwan Strait citing the United Nations Convention on the Law of the Sea (UNCLOS) when the US has not even ratified the universal convention.
Why drones are no big deal
Regarding drones, India has agreed to buy 31 General Atomics (GA) MQ-9B (Sea Guardian and Sky Guardian) armed and unarmed drones under Foreign Military Sales (FMS) for the price of approximately US$3 billion. The offset amount of this deal will be used by GA to establish an MRO facility in India. These drones (also called Predators and Reapers) are extremely expensive, slow-moving, and completely out of sync with present and future trends in warfare. On the positive side, these drones carry big payloads, and have good range and mission capability since they are equipped with electro-optic video cameras, laser designators, good communication relay with ground stations, good electromagnetic systems, and signal intelligent equipment. They also have endurance of 40 hours. On the downside, with slow speed they are vulnerable and can be shot down by enemy air defence systems. Moreover, they lack autonomy (AI), are not stealthy and, most of all, require a lot of people on the ground to operate them.
Representative image of a MQ-9 drone. Photo: Twitter
In his book Army of None, US analyst Paul Scharre writes, ‘Predator and Reaper drone operations require seven to ten pilots to staff one drone orbit of 24/7 continuous around-the-clock coverage over an area. Another 20 people per orbit are required to operate the sensors on the drone, and scores of intelligence analysts are needed to sift through the sensor data. In fact, because of these substantial personnel requirements, the US Air Force has a strong resistance to calling these aircraft unmanned.’
The irony is that when these UAVs had a lucrative market in the early 2000s, US laws did not allow their sale. Now, when the US has approved their export, many more cost-effective and better-performing drones with AI options are available. Incidentally, China is a leading exporter of military drones. There can only be two reasons for India buying these outdated and expensive drones: US pressure and the Indian military’s tendency to conflate capability with sophisticated weapon platforms.
What iCET and INDUS-X really represent
The US pressure for this purchase is for its Maritime Domain Awareness (MDA) under the Indo-Pacific partnership which was announced in Tokyo in 2022. Under this, the Quad members are to do data collection and sharing of Intelligence, Surveillance, and Reconnaissance (ISR) feeds with partners in Southeast Asia and Pacific nations. India, given its location, has a special responsibility which is evident from two things: It is the only country which has operational interaction with three US theatre commands namely, INDOPACOM, Central Command, and Africa Command. And the Indian Navy’s Gurgaon-based Information Management and Analysis Centre (IMAC), whose software has been supplied by US companies CISCO and Raytheon, will be the nerve centre of the MDA project. To keep data cyber-secure, new undersea cables will be laid connecting all Quad members.
To understand the iCET framework and the INDUS-X scheme – which have been touted as the biggest US offering to India since the civil-nuclear deal – it is essential to first understand the context. Following the 2012 discovery of Deep Learning which revolutionised AI, and conscious that most emerging technologies including AI were being incubated in the commercial sector, the Pentagon in 2015 opened its outpost called Defence Innovation Unit (DIU) in Silicon Valley. Staffed with mostly civilian software experts and a few retired and active-duty military officers, the DIU was to help identify advanced commercial technologies for military use.
Taking a cue from the Pentagon’s DIU, the Indian defence ministry, in April 2018, launched the Innovation for Defence Excellence (iDEX) scheme under the defence secretary for incubation of new age technologies development in defence and aerospace from within the industry, start-ups, civilian laboratories, academia and so on. Unfortunately, iDEX has not delivered for four reasons: it is headed by bureaucrats with little domain knowledge, many start-ups that get good funding are owned by former military officers with questionable links in the system, deserving start-ups with few resources are bought by the Defence Research and Development Organisation, and projects are time-bound, which stymies creativity, ideas, and innovations.
Worse, there is little clarity on new-age commercial technologies prioritisation and how to harness them for military use. For instance, there are 20 advanced commercial technologies to choose from for military use. These are AI (where all new age technologies converge), 5G wireless and advanced networking, sensors with edge computing, Internet of Things, biotechnology, robotics and autonomy, semiconductors, blockchain, virtual reality (for simulation), metaverse, star-link internet, natural languages processing, space, cyber, advanced manufacturing, energy storage technologies, quantum computing, brain-computer interface, genomics, and exoskeletons.
Meanwhile, aware that talent is the most important pillar of AI alongside data, hardware (microelectronics), and software (algorithms), and with many talented Chinese AI researchers leaving the US for China following the tech war, the US, which attracts talent from across the world (mostly Chinese and Indians) to remain the global innovation hub, seemed to be losing the competition for talent to China.
Against this backdrop, the Indian and US National Security Advisors met in January 2023 to launch the ICET framework to be the bridge between commercial and defence sectors. Under the ICET, the two sides launched the INDUS-X scheme chaired jointly by the Pentagon and India’s iDEX to identify Indian defence start-ups which will work with the US start-ups organised by the civilian US-India Business Council (USIBC) to design, prototype, test, and produce commercial technologies with application in ‘integrated deterrence’ network. While in theory, the money for identified start-ups will come from a joint development fund, in practice they will be funded by the Pentagon. This way, the US military will be able to match, if not beat, the PLA’s 2035 deadline of robotic war (called intelligentised war) and make up for its talent shortage using Indian researchers to meet the Chinese challenge of becoming the world’s primary AI innovation centre by 2030.
It is for this reason that the US is easing H1B and L1 visas procedures for Indian STEM (science, technology, engineering, mathematics) graduates and research scholars to work in the US. This is a good thing for Indian scholars and start-ups who will gain knowledge by working in world-class innovation hubs. Of course, it would have been better if the Indian civilian start-ups could work directly with the DIU (instead of through the iDEX and the Pentagon) for the understanding of all advanced commercial technologies.
However, for India which prided itself on its strategic autonomy in foreign policy, allowing Indian shipyards to emerge ‘as a hub for maintenance and repair of forward deployed (in the IOR) US navy assets (navy ships and naval fighters)’ is too big a price to pay for Modi’s rousing welcome in the US. Taking forward the Logistics Exchange Memorandum of Understanding (LEMOA) signed by the Modi government in 2016, which allowed re-fuelling and other turn-around facilities to the US naval assets on a case-to-case basis, the present arrangement will permit the US to upgrade Indian shipyards to berth US warships for long periods. Shorn of rhetoric, India has agreed to provide military basing to the US military.
Also Read: Six (of Many) Reasons Why PM Modi’s Words on Democracy in the White House Ring Hollow
The impact on China
The big question now is: How will China assess India’s tight embrace of the US? China will conclude that normalisation of relations with the Modi government will not be possible since India has sacrificed its strategic autonomy to accommodate US military interests in the IOR.
Worse, India has become the US’s first line of offence against Chinese interests and infrastructure in the South Asian region (by denouncing its Belt and Road Initiative as a debt trap). Moreover, in cahoots with the US, India will be seen to threaten China’s commerce and trade worth over US$4 trillion annually which passes through the 3,000 nautical miles IOR from the Strait of Hormuz to the Strait of Malacca. This, at a time when the Chinese deterrence in the IOR is a decade away.
Xi Jinping’s directive to the PLA was to prepare capabilities (deterrence) to meet the (US) challenge in West Pacific and the Indian border by 2027, and across Asia Pacific (including IOR) by 2035. The PLA has already achieved deterrence (and capabilities to fight if deterrence failed) ahead of its timeline in the West Pacific and is working on the entire region.
Chinese Presindent Xi Jinping. Photo: Paul Kagame/Flickr CC BY-NC-ND 2.0
China’s deterrence in the West Pacific where it faces challenges in the Taiwan Strait and the South and East China Sea is based on the combination of its military power and economic power (its intense trade with ASEAN and US allies like Japan and South Korea). Given the advantage of its geography, and the formidable Anti-Access Area Denial (A2AD) firewall that the PLA has created against the US, China refused Antony Blinken’s request in Beijing for opening bilateral military communication to avert a crisis in the region. While the stated reason by China for doing this are the US sanctions on its defence minister, Gen. Li Shangfu, the actual reason is to discourage US freedom of navigation and air patrols in the waters and air space in the Chinese backyard. Much in line with the US’s Monroe Doctrine – which disallows the presence of outside powers in the Western hemisphere – China wants US military activities in West Pacific to end.
Meanwhile, aware of its eroded deterrence in the West Pacific, the US military is reinforcing its regional alliances, seeking a global role for NATO and strengthening its military bases in Hawaii and Guam.
Chinese deterrence outside the West Pacific will be based on its economic deterrence anchored on BRICS and BRI, and its military deterrence anchored on the Shanghai Cooperation Organisation and cooperative security with nations onboard the BRI. Since prosperity is at the vanguard of the BRI, Xi Jinping has announced that the third BRI Forum will be held in Beijing this year. The actual dates of the Forum perhaps await progress on Xi’s pet project of ‘Xiongan (meaning brave and peaceful) New Area’, 100km outside Beijing which is being called the vice-capital of China. Named the ‘plan of a thousand years’ by Xi, this city, meant to de-congest Beijing by moving many governments, financial, medical, academic, and military headquarters here, is being built with cyberspace software connectivity (part of the Digital Silk Road, a sub-set of the BRI) using industrial internet technologies like blockchain, 5G, IoT, big data and so on. This smart city will become the blockchain hub in China with blockchain-based services. Given this, it will be a good idea for Xi to showcase this smart city close to Beijing to the Forum delegates for them to get a sense of how the new phase of BRI (cyberspace software connectivity) which will likely figure in Xi’s address, would usher in prosperity by industrial internet.
China’s economic deterrence for BRI nations will get delayed owing to the chip war with the US. For example, China has been denied advanced chips which are required in AI and data centres for computing, storage, and for servers. Slowing economic deterrence will affect military deterrence which is based on cooperative security (PLA working with host BRI nation’s security forces for the protection of Chinese infrastructure, interests, and people).
Given the likelihood of China’s military deterrence lagging behind the US – which is supported now by India in South Asia and the IOR – Beijing might decide to up the ante on its border. Since China sees the boundary problem with India as a dispute regarding its sovereignty, it may decide to upend the US’s growing integrated deterrence in the region by a short and decisive war with India. In that sense, Modi’s US embrace has increased India’s vulnerabilities.
Pravin Sawhney is the author of The Last War: How AI Will Shape India’s Final Showdown With China.


"""

In [117]:

# #### number of points for above news has been obtained in 2 parts as there is a limit on numbe rof tokens in GPT4


# The given article discusses Prime Minister Narendra Modi's visit to the US and its potential implications for India's strategic and security position. Analyzing the content, one can identify several points that are presented either in a positive or negative light in relation to the broader context of India's relationship with the US and the geopolitical scenarios in the region.

# **Positive Points:**

# 1. **Geopolitical Synergy**: Both the US and India have converging interests - the US needs India for global geopolitics, while India looks to the US for regional geopolitics.

# 2. **Collaboration on Defense Technologies**: The creation of the India-United States Defence Acceleration Ecosystem (INDUS-X) and the Innovation on Critical and Emerging Technologies (ICET) framework indicates a commitment to upgrade and integrate India's defense capabilities within the broader US defense network.

# 3. **Capability Upgrade through Deals**: The deal with General Electric (GE) to supply engines for the Indian Air Force (IAF) Light Combat Aircraft (LCA) programs is a step towards modernizing India's military equipment.

# 4. **Drones Acquisition**: The procurement of the MQ-9B drones from General Atomics offers advanced capabilities like long endurance, substantial payload capacity, and sophisticated communication and intelligence equipment.

# **Negative Points:**

# 1. **Compromised Strategic Autonomy**: The article suggests that the closer military partnership with Washington might limit India's strategic autonomy and its ability to independently navigate geopolitical challenges.

# 2. **Questionable US Deterrence Model**: The US's deterrence model, especially its stance against China, might not be suitable in the current geopolitical environment.

# 3. **Potential Global Economic Instability**: The US's attempt to decouple from China's economic system might lead to increased geopolitical tensions and potential global recession.

# 4. **US Dependence on India against China**: The US's strategy to build India as a military counterweight to China in the Indian Ocean Region (IOR) might be placing undue pressure and expectations on India.

# 5. **Transactional Nature of Relationship**: Concerns that India might give up more, including its strategic autonomy, than it receives in return from the US.

# 6. **Shift Away from Traditional Partners**: The US's attempt to wean India away from traditional allies like Russia might have long-term strategic implications.

# 7. **Strategic Isolation from Neighbors**: By aligning more with the US, especially in terms of technological standards, India might find itself isolated from its South Asian neighbors, most of whom are part of China's Belt and Road Initiative (BRI).

# 8. **Limited Technology Transfer in Deals**: For instance, in the GE deal, there is no transfer of significant technology, and India mainly gets assembly rights.

# 9. **Questionable Procurement Decisions**: The acquisition of certain equipment, like the MQ-9B drones, might not align with the future of warfare, and there are concerns over their utility and relevance.

# 10. **Potential US Sanctions**: The article raises concerns about the reliability of the US as a partner, citing instances where the US has overlooked legalities, hinting at the possibility of sanctions even after defense deals.

# Based on this analysis, there are 4 positive points and 10 negative points presented in the context of Modi's US visit and its implications for India's geopolitical and strategic positioning.


# part 2

# Here are the positive and negative points extracted from the provided content:

# **Positive Points**:
# 1. The US and Quad members aim to enhance Maritime Domain Awareness (MDA) by improving data collection and intelligence sharing in the Indo-Pacific.
# 2. India holds a pivotal role due to its operational interactions with three US theatre commands.
# 3. The IMAC, supported by US companies, will be the core of the MDA project.
# 4. New undersea cables will enhance data security among Quad members.
# 5. The Pentagon's Defense Innovation Unit (DIU) in Silicon Valley aims to harness advanced commercial technologies for military use.
# 6. India launched the Innovation for Defence Excellence (iDEX) scheme to foster technology development in defense and aerospace.
# 7. The ICET framework and INDUS-X scheme aim to bridge the commercial and defense sectors, connecting Indian and US start-ups.
# 8. The US is easing visa procedures for Indian STEM graduates, which benefits Indian scholars by giving them opportunities in world-class innovation hubs.

# **Negative Points**:
# 1. The iDEX scheme has been underwhelming due to lack of domain expertise, questionable links of funded start-ups, buyouts of promising start-ups by DRDO, and time-bound projects that limit creativity.
# 2. There's a lack of clarity on how to prioritize and harness new-age technologies for military use.
# 3. The US seems to be losing the AI talent race to China.
# 4. There's a sentiment that India has sacrificed some of its strategic autonomy by allowing its shipyards to serve US navy assets extensively, moving beyond the initial scope of the LEMOA agreement.
# 5. India's closer ties with the US might complicate its relations with China, as China might perceive India as sacrificing its strategic autonomy to accommodate US military interests.
# 6. India's stand against China's Belt and Road Initiative might be seen as antagonistic.
# 7. China's plans for military deterrence in the Indo-Pacific region are projected to lag behind, and Beijing might escalate tensions on its border with India in response.
# 8. Overall, the author suggests that India's strengthened alliance with the US may increase its vulnerabilities against China.

# This division is based on the perspective provided by the article. Different analyses might interpret some points differently.

In [118]:
num_pos_pts1 = 12
num_neg_pts1 = 18

In [119]:
url7 = 'https://www.financialexpress.com/business/defence-exclusive-us-india-strategic-partnership-forum-ceo-says-modis-us-visit-paves-way-for-robust-deals-and-strategic-partnership-3180844/' 


sentence7 = """
Exclusive: US-India Strategic Partnership Forum CEO says Modi’s US visit paves way for robust deals and strategic partnership
The State Visit highlighted important steps in trade resolutions as the U.S. and India agreed to resolve six separate trade disputes at the World Trade Organization, including President Trump’s previous tariffs on steel and aluminum and India’s retaliatory duties.

Prime Minister Narendra Modi’s recent State Visit to the US has been a resounding success, garnering significant attention as India holds the G20 presidency amidst the ongoing Russia-Ukraine war, impacting global food and energy security.

Dr Mukesh Aghi, President & CEO of the US-India Strategic Partnership Forum (USISPF), shared his insights with Huma Siddiqui on the visit and the inked deals, shedding light on the visit’s significance and the robust agreements achieved in critical and emerging technologies and defence.

The visit was symbolic and only the third state visit for an Indian leader, the last being in 2009 and the third state visit hosted by the Biden administration. So, this clearly reflected the importance of India and India’s place in the world for the Biden Administration.

But beyond the ceremonial festivities, the visit was extremely successful and elevated the strategic partnership to the next level with robust agreements in critical and emerging technologies and defense particularly. The deals pertaining to jet engines, drones, semiconductor investments, working together from the seas to the stars from joint-space cooperation, and enhanced underwater domain awareness. The priorities are also towards a clean energy future, building new resilient supply chains as India ramps up its manufacturing potential. The US-India partnership remains committed to a peaceful end to hostilities in Eastern Europe and securing a free and open Indo-Pacific.

The priority is to secure a clean energy future and one where American investment in India’s clean energy potential will wean India of needing Russian energy imports. Presently India imports oil from Russia factoring in India’s energy security priorities. But gradual steps are being made and the Strategic Clean Energy Partnership between India and the United States. These are the steps in the right direction, as it works towards energy security and electrification. It’s a gradual process and needs to be done in phases. Washington understands New Delhi’s energy priorities and understands New Delhi is firmly aligned with Washington’s geopolitical priorities in the Indo-Pacific as Beijing increases its hostilities. So, India’s energy imports are purely for its own energy security and will not dampen the strong partnership with Washington.


India is the fastest growing economy and now the largest populous nation on the planet. India needs to add 12-15 million jobs a year in order to remain competitive and add new jobs to those joining the workforce. Services alone can’t solve this and need to focus on manufacturing sectors as well. For this to happen, the bulk of the investments need to come in from American companies and investors. The United States is the largest economy and the synergy between the US and India is strong in areas like tech, energy, defense, and healthcare. Prime Minister Modi’s government has worked to improve the business environment. Examples like the Air India-Boeing deal, and Apple opening its first retail store in India are strong examples of the strength of the commercial relationship.

These are also examples of the growing strength of the manufacturing economy, as it sends a signal to American high-tech multinationals to come and “Make in India”. India is working to build its manufacturing economy, especially for cutting-edge tech products. The other priorities for both Washington and New Delhi are to build supply chain resilience and diversify supply chains away from China as companies in boardrooms look at a China-plus-one strategy.

The State Visit highlighted important steps in trade resolutions as the U.S. and India agreed to resolve six separate trade disputes at the World Trade Organization, including President Trump’s previous tariffs on steel and aluminum and India’s retaliatory duties. While there is no existing trade agreement, the signs are still auspicious as India and the US have restarted the Trade Policy Forum (TPF) to solve existing issues. One needs to understand that diplomatic ties are not concurrent with trade relations. Even the US and Canada which share excellent diplomatic ties can spar over trade issues. India has signed several FTAs with countries like the UAE and Australia and is working on trade deals with the UK, so the current government is removing some of the previous protectionist tendencies that were evinced in India. We are seeing gradual opening with mangoes, pomegranates from India coming to the US and pork and hay coming from the US.

New Delhi has also agreed to remove retaliatory tariffs on certain US products, agricultural primarily in chickpeas, lentils, almonds, walnuts, and apples. So, as you can see, there is no boiling the ocean strategy, but gradual phases of removing existing barriers that will pave the way for a more auspicious trade environment and then perhaps we can speak of larger trade deals





"""

In [120]:
# The text titled "Exclusive: US-India Strategic Partnership Forum CEO says Modi’s US visit paves way for robust deals and strategic partnership" provides insights into the recent state visit of Prime Minister Narendra Modi to the U.S. Here's a breakdown of the positive and negative points:

# **Positive Points**:
# 1. The visit highlighted steps in trade resolutions between the U.S. and India.
# 2. The state visit by PM Modi garnered significant attention globally, especially during sensitive times such as the Russia-Ukraine war.
# 3. The visit emphasized robust agreements in critical and emerging technologies and defense.
# 4. Symbolic importance: Only the third state visit for an Indian leader and reflects the importance of India for the Biden Administration.
# 5. Elevated the strategic partnership with robust agreements in multiple sectors.
# 6. Commitment to a peaceful resolution in Eastern Europe and a free and open Indo-Pacific.
# 7. Emphasis on a clean energy future and reducing dependence on Russian energy imports.
# 8. Mutual understanding between Washington and New Delhi about each other's priorities.
# 9. India's growth and the importance of job creation to support its booming population.
# 10. Strong commercial relationship examples such as the Air India-Boeing deal and Apple's retail store opening in India.
# 11. Emphasis on strengthening the manufacturing economy in India.
# 12. Prioritizing supply chain resilience and diversifying away from China.
# 13. Resolving trade disputes at the World Trade Organization.
# 14. Reinstatement of the Trade Policy Forum to solve existing issues.
# 15. India’s openness to trade by signing FTAs with various countries.
# 16. Gradual phases of removing trade barriers for a more auspicious trade environment.

# **Negative Points**:
# 1. Despite the significant trade resolutions, there's no existing trade agreement between the U.S. and India.
# 2. India's past protectionist tendencies have caused some concerns.

# Based on the provided text, there are 16 positive points and 2 negative points.

In [121]:
num_pos_pts7 = 16
num_neg_pts7 = 2

In [122]:
url8 = 'https://www.cnn.com/2023/06/21/politics/joe-biden-narendra-modi-visit/index.html' 


sentence8 = """

Welcoming Indian Prime Minister Narendra Modi to the White House this week for a state visit – the most elevated form of American diplomacy – required President Joe Biden to make certain trade-offs and Modi to step out of his comfort zone, for at least a few minutes.
Modi, massively popular in India, has demonstrated a drift toward authoritarianism that has worried the West. He’s cracked down on dissent, targeted journalists and introduced policies that human rights groups say discriminate against Muslims.
Yet Modi and India, the world’s largest democracy, also represent a lynchpin in Biden’s strategy in Asia. The country recently surpassed China to become the most populous country on Earth. No major global challenge, from climate change to advances in technology, can be addressed without India’s buy-in, in Biden’s view. And in an era of growing tensions between the US and China, there are few partners that Biden is more eager to cultivate.
That, according to officials, was the rationale behind inviting Modi for a state visit, only the third of Biden’s presidency so far.
And so on Thursday the prime minister was welcomed to the White House with the highest trappings of American friendship: Marching troops on the South Lawn, extensive Oval Office talks and a state dinner in the evening, complete with a chef who specializes in plant-based cuisine to accommodate Modi’s vegetarian diet.
“I’ve long believed the relationship between the United States and India … will be one of the defining relationships of the 21st century,” Biden said during an elaborate welcome ceremony during which he hailed the friendship between the US and India.
Modi takes reporters’ questions in remarkable moment
Later, when the two leaders held a joint news conference that had been subject of intense negotiations between American and Indian officials, it was only Biden who called on reporters.
But Modi, for one of the only times since he assumed power, also responded to a question about his crackdowns on dissent and treatment of religious minorities.
“There’s absolutely no space for discrimination,” Modi said through a translator. ‘When you talk of democracy, if there are no human values and there is no humanity, there are no human rights, then it’s not a democracy.”
It was a remarkable moment for a leader who rarely holds news conferences, takes questions from the press or sits for international interviews. Press freedom groups say he’s overseen a crackdown on reporting.
The format of the news conference was only agreed upon after lengthy, delicate negotiations between the two sides. Indian officials initially balked at the White House’s insistence at holding one, two US officials familiar with the matter said.
Biden, in his own answer on Modi’s human rights record, underscored the shared values between the US and India.
“The prime minister and I had a good discussion about democratic values,” he said. “That’s the nature of our relationship, we’re straightforward with each other and we respect each other.”
“I believe that we believe in the dignity of every citizen, and it is in American’s DNA, and I believe in India’s DNA,” the president told reporters Thursday, adding that the two countries’ commitment to democracy “makes us appealing partners and enables us to expand democratic institutions across around the world.”
An important international relationship
As he welcomed Modi to the White House earlier in the day, Biden sought to underscore the importance of shared values in the relationship moving forward.
“As democracies, we can better tap into the full talent of all of our people, and attract investments as true and trusted partners as leading nations, with our greatest export being the power of our example,” Biden said from a podium on the South Lawn. “Equity under the law, freedom of expression, religious pluralism and the diversity of our people. These core principles have endured and evolved even as they have faced challenges throughout each of our nation’s histories and fuel our strength, depth and future.”
President Joe Biden speaks with India's Prime Minister Narendra Modi during a State Arrival Ceremony on the South Lawn of the White House in Washington, Thursday, June 22, 2023. (AP Photo/Andrew Harnik)
Andrew Harnik/AP
The meetings Thursday produced agreements on technology and defense cooperation and highlighted Biden’s efforts to shore up relationships in a region grappling with an increasingly aggressive Beijing.
Also up for discussion was Russia’s continuing war in Ukraine, a conflict on which India has not taken a definitive side. New Delhi’s continued purchase of Russian oil has helped prop up Moscow amid withering global sanctions.
“Fundamentally, we believe that the long-term trajectory of the US-India relationship is built on the notion that two democracies with shared value systems ought to be able to work together,” US National Security Adviser Jake Sullivan told a group of reporters ahead of the visit.
“That’s a long view. That’s a view rooted in our bet on the relationship between the people of the United States of America and the people of India,” Sullivan continued. “Part of what will be lifted up and celebrated in this visit will be those deep people and people ties.”
U.S. President Joe Biden and first lady Jill Biden welcome Prime Minister of India Narendra Modi to the White House in Washington, U.S., June 21, 2023. REUTERS/Elizabeth Frantz
Elizabeth Frantz/Reuters
Courting a controversial leader
Modi is hardly the first leader with authoritarian tendencies to be invited for a state visit at the White House. President Barack Obama hosted Chinese President Xi Jinping in 2015, shortly after he assumed power in Beijing.
And Biden is not the first president to embrace Modi. Alongside President Donald Trump, the Indian prime minister attended a massive “Howdy Modi” rally in Houston for members of the Indian diaspora. Modi returned the favor in Ahmedabad with a “Namaste Trump” event held in the world’s largest cricket stadium.
US President Joe Biden and India's Prime Minister Narendra Modi walk through the Colonnade to the Oval Office of the White House in Washington, DC, on June 22, 2023. 
Evan Vucci/Pool/AFP/Getty Images
Other world leaders have offered a similar welcome. French President Emmanuel Macron will host Modi as the guest of honor at this year’s Bastille Day parade.
But Biden has made as a centerpiece of his foreign policy the battle between “democracy and autocracy,” a thematic backdrop perhaps nowhere more relevant than in India.
In March, the former leader of India’s main opposition political party, Rahul Gandhi, was disqualified as a lawmaker a day after he was handed a two-year jail sentence for defamation in a ruling his supporters called politically motivated.
Modi’s ruling Bharatiya Janata Party (BJP) has come under scrutiny from rights groups and opposition lawmakers for its increasingly strident brand of Hindu nationalist politics and an ongoing crackdown on dissent.
Modi himself was denied a visa to the United States in 2005 because of his alleged role in anti-Muslim violence three years earlier in Gujarat state, where he was chief minister. More than 1,000 people were killed, mostly Muslims. A Supreme Court-ordered investigation in India absolved him of blame.
A wide array of human rights groups, lawmakers and dissidents have made clear they want Biden to raise his concerns on Modi’s record when he’s at the White House. A group of more than 70 congressional Democrats wrote a letter to Biden this week urging him to bring up human rights concerns when he meets with Modi.
“You have once again made respect for human rights, press freedom, religious freedom, and pluralism core tenets of American foreign policy. Moreover, these tenets are necessary to the functioning of true democracy. In order to advance these values with credibility on the world stage, we must apply them equally to friend and foe alike, just as we work to apply these same principles here in the United States,” the lawmakers wrote.
 U.S. President Joe Biden (R) and Indian Prime Minister Narendra Modi meet in the Oval Office at the White House on June 22, 2023 in Washington, DC. Biden and Prime Minister Modi will later participate in a joint press conference and a state dinner in the evening. Biden is the first U.S. president to invite Modi for an official state visit. 

The letter was spearheaded by Rep. Pramila Jayapal, a Washington state Democrat, and Sen. Chris Van Hollen, a Maryland Democrat. Jayapal said in a tweet about the letter, “We must ensure freedom of the press, religious tolerance, internet access, & the diversity of political thought.”
Six Democratic lawmakers boycotted Modi’s address to Congress Thursday. Reps. Rashida Tlaib, Ilhan Omar and Alexandria Ocasio-Cortez pointed before the address to Modi’s treatment of Muslims.
Grappling with China’s influence
Thursday’s state visit included announcements in the technology space as well as defense cooperation, including on defense sales, production and technology collaboration, an administration official said.
American defense trade with India had boomed in the past 15 years, and US officials have noted India is diversifying away from relying on any one nation, like Russia, for arms.
“You’ll see major cancellations of defense system purchases from Russia, in part because we know Russia can’t actually provide it, that Russia is consuming its own defense production very quickly in this terrible war in Ukraine,” a senior administration official said. “And everyone around the world who buys Russian equipment is worried about whether they can get spare parts and then new systems, given the supply chain problems that Russia is experiencing.”
Among the announcements Thursday was that India has committed to the purchase of MQ-9B SeaGuardian drones, deepening the two nations’ military ties amidst the backdrop of an increasingly bellicose China.
Biden and Modi planned an agreement between US manufacturer GE and the state-owned Hindustan Aeronautics to jointly produce F414 jet engines in India.
Other announcements included India joining the US-led Artemis Accords, an international cooperative agreement for space exploration, mounting a joint-mission with NASA to the International Space Station in 2024 and a commitment from US-based semiconductor producer Micron Technology to break ground on a $2.75 billion new semiconductor assembly and test facility in India.
The discussions between the two men also touched on enhancing educational exchanges and were expected to feature new steps forward on visas and diplomatic presence in each country.
They also discussed agenda issues for the G20 summit, which India is preparing to host in September.
Biden’s efforts to deepen ties with India comes as the two countries grapple with the growing influence of China. The president has met with Modi on several occasions, including through meetings of the Quad leaders.
He said at a fundraiser this week that his revival of the four-way alliance had upset Xi.
“He called me and told me not to do that because it was putting him in a bind,” Biden said, referring to the Chinese president. “We’re just trying to make sure the international rules of air and sea lanes remain open.”
Sullivan, however, said Thursday’s visit at the White House was “not about China,” though “the question of China’s role in the military domain, the technology domain, the economic domain will be on the agenda.”
This story has been updated with additional reporting.
CORRECTION: A previous version of this story mischaracterized how often Modi engages with the media. News conferences and international interviews are rare but they have occurred.
CNN’s Donald Judd contributed to this report.

"""

In [123]:
# Here's a breakdown of the positive and negative points based on the provided text:

# **Positive Points**:
# 1. Narendra Modi's state visit represents the highest form of American diplomacy, signifying a special and important relationship between the two countries.
# 2. India has become a pivotal partner in Biden’s strategy in Asia.
# 3. India recently surpassed China as the most populous country, and Biden believes addressing major global challenges requires India's collaboration.
# 4. The state visit had elaborate ceremonies, indicating a strong partnership and respect between the two nations.
# 5. Biden emphasized the defining relationship of the 21st century between the US and India.
# 6. Modi responded to concerns about his crackdowns on dissent and treatment of religious minorities.
# 7. Biden spoke positively about shared democratic values between the US and India.
# 8. There were agreements on technology and defense cooperation between the two countries.
# 9. The US and India emphasized the importance of their shared values in global leadership.
# 10. The US acknowledges the importance of India in the region, especially given the challenges from an increasingly aggressive Beijing.
# 11. The article mentioned announcements about technology and defense collaborations that are beneficial for both nations.
# 12. India's commitment to various initiatives, like space exploration in partnership with the US.
# 13. Discussion on enhancing educational exchanges and positive steps regarding visas and diplomatic presence.

# **Negative Points**:
# 1. Narendra Modi has shown tendencies towards authoritarianism, which concerns the West.
# 2. Modi's policies have been perceived as discriminatory against Muslims.
# 3. Only Biden called on reporters during the joint news conference, and Modi rarely holds news conferences.
# 4. Modi's government has been criticized for suppressing press freedom.
# 5. The news conference's format had to be delicately negotiated due to initial reluctance.
# 6. Modi was previously denied a visa to the US because of his alleged role in anti-Muslim violence.
# 7. There were calls from human rights groups for Biden to address concerns about Modi's record.
# 8. More than 70 congressional Democrats urged Biden to bring up human rights concerns when meeting Modi.
# 9. Six Democratic lawmakers boycotted Modi's address to Congress, pointing to his treatment of Muslims.
# 10. India hasn't taken a clear stance on Russia's war in Ukraine and continues to purchase Russian oil.
# 11. Modi's past includes controversies such as being denied a US visa due to alleged involvement in anti-Muslim violence.
# 12. Some lawmakers and rights groups have criticized Modi's party for its Hindu nationalist politics and ongoing crackdown on dissent.

# This text presents a mix of positive and negative points, offering a balanced view of the current relationship between the US and India, as well as some historical context about PM Narendra Modi.

In [124]:
num_pos_pts8 = 13
num_neg_pts8 = 12

In [125]:
url9 = 'https://www.aljazeera.com/news/2023/6/21/whats-behind-indian-prime-minister-narendra-modis-us-visit' 


sentence9 = """

What’s behind Indian Prime Minister Narendra Modi’s US visit? | Narendra Modi News | Al Jazeera
|Narendra ModiWhat’s behind Indian Prime Minister Narendra Modi’s US visit?The US is seeking stronger ties with India, which it sees as a vital ally in efforts to contain China’s rise.US President Joe Biden speaks with Indian Prime Minister Narendra Modi at the G20 summit in November [File: Prasetyo Utomo/G20 Media Center/Handout via Reuters]By Brian OsgoodPublished On 21 Jun 202321 Jun 2023The administration of United States President Joe Biden is scheduled to host Indian Prime Minister Narendra Modi during an official visit later this week, as the two countries strengthen their ties amid shared antipathy towards China’s growing influence.
The White House will hold a state dinner in Modi’s honour on June 22, a sign of the burgeoning relationship between the two powers who have stepped up cooperation in areas such as trade and arms sales.
In a press release, the Biden administration said that a recent trip to New Delhi by National Security Advisor Jake Sullivan “underscored the dynamism of the US-India partnership in advance of Prime Minister Modi’s historic official state visit next week to the United States”.
But human rights groups say the celebratory dinner is a de facto endorsement of India’s far-right turn under Modi’s leadership — and undermines the Biden administration’s stated goal of emphasising human rights and democracy in its foreign policy.
During Modi’s tenure, India’s Muslims and other minorities have experienced an uptick in violence and repression as the government leans into a form of Hindu nationalism known as Hindutva. Modi has also been criticised for seeking to consolidate power and crack down on dissent.
“A state dinner is a special occasion; it’s not something that just any foreign leader receives,” Edward Mitchell, the deputy executive director of the Council on American-Islamic Relations (CAIR), told Al Jazeera over a phone call.
“How can the White House honour a leader who is an open anti-Muslim bigot, a right-wing ideologue who censors journalists and turns a blind eye to lynchings? You can work with India and Modi without taking this extra step to celebrate him.”
Confronting China
While advocacy groups have called for greater scrutiny of India’s human rights record, foreign policy experts say the Biden administration is primarily interested in the country as a potential counterweight to China, which the US sees as its most formidable global competitor.
Sarang Shidore, Director of Studies and Senior Research Fellow at the Quincy Institute, a US-based think tank, told Al Jazeera that he believes the US-India relationship will continue to grow as long as their shared concern over China remains in place.
“The United States does not conduct its foreign policy based on democracy and human rights. It conducts its foreign policy based on its interests, as all states do,” Shidore said.
China’s growing military power and assertive territorial claims have become a source of concern for nearby Asian countries such as the Philippines, Vietnam, Japan and India. The US has worked to fashion alliances with many of those countries in an attempt to contain China’s expanding influence.
“No question, there is increased Chinese nationalism,” said Shidore. “And China sees India as an increasing challenge due to its relationship with the US.”
Expanding ties
That was not always the case. During the Cold War, India’s relations with the US were often frosty. The country had cultivated close ties with the USSR and helped spearhead the Non-Aligned Movement, an organisation of countries that rejected pressures to join either pro-US or pro-Soviet blocs.
For its part, the US was a key ally of Pakistan. And by the early 1970s, the administration of US President Richard Nixon started to build a cooperative relationship with China, as an attempt, in part, to place pressure on the USSR.
But as the Cold War ended and China’s economic rise became a preoccupation of US foreign policy, India — with its size and economic heft — started to be seen as a key regional ally.
Despite its improved ties with the US, however, India has continued to resist what it sees as a false choice between the US and countries like President Vladimir Putin’s Russia.
But as Russia wages war in Ukraine and Washington seeks to isolate Moscow economically and diplomatically, that balancing act has become more difficult for India to maintain.
While India has increased its purchase of weapons from countries such as France and the US and recently agreed on a roadmap to increase cooperation with the US defence industry, it remains the world’s largest importer of Russian arms.
India has also joined China in buying up Russian oil at discounted prices, while the US and the European Union angle to limit Russia’s power in the global energy market.
But Shidore said that India’s status as a central player in Washington’s Asia strategy gives it significant leverage. Its ties to Russia are not likely to get in the way of its relationship with the US, he explained.
“India has played this quite well, playing Russia and the US off each other, and has benefitted in the process,” he said. “A country like India, which has such a strong convergence with the US on China, can create major spaces where it will differ very strongly from the US and can ride that out.”
Consolidating control
While US relations with allies such as Saudi Arabia and Israel have come under political scrutiny in recent years, Modi’s trip to the US has been welcomed with bipartisan support. An a joint letter inviting Modi to address Congress during his visit, members of the US House of Representatives and Senate have hailed the visit as a sign of the “enduring friendship” between the two countries.
“During your address, you will have the opportunity to share your vision for India’s future and speak to the global challenges our countries both face,” the letter reads.
However, Modi’s human rights record has not gone entirely unremarked. On Tuesday, a group of more than 70 lawmakers from the US House and Senate penned a letter to the Biden urging him to discuss concerns about religious freedom and journalistic expression in his talks with Modi.
The Muslim rights group CAIR, meanwhile, has issued a statement calling on the White House to drop its plans for a state dinner.
Modi’s high-profile reception in the US is a far cry from what he experienced before he was first elected prime minister in 2014. Prior to becoming India’s leader, Modi had been banned from entering the US due to allegations that he turned a blind eye to anti-Muslim violence in the western Indian state of Gujarat in 2002, when he was the province’s chief minister.
The deadly 2002 riots were the subject of a BBC documentary that Modi tried to ban in January, invoking his emergency powers as prime minister. The rights groups Amnesty International and Human Rights Watch screened the documentary in Washington on Tuesday.
Human rights groups say the move to ban the film was representative of a larger effort under Modi to stifle dissent and exercise control over independent institutions. They also accuse his government of pursuing an agenda influenced by far-right Hindu nationalism.
In an annual report on religious freedom in May, the US State Department expressed concern about the situation in India, noting that there were “open calls for genocide against Muslims”, lynchings and “attacks on places of worship”.
In May, for the fourth year in a row, the US Commission on International Religious Freedom called on the State Department to designate India as a “country of particular concern”.
In some states controlled by Modi’s Bharatiya Janata Party (BJP), politicians who lean into violent anti-Muslim rhetoric face little reprimand. Some Hindu nationalist paramilitaries have also waged a campaign against interfaith marriages, which they portray as an effort to dilute the Hindu population and win converts to Islam through “love jihad”.
Shidore, however, said that Modi’s human rights record has been quickly “papered over” and is unlikely to prevent cooperation with the US, so long as China remains a serious global competitor.
“The United States”, he said, “has set human rights issues aside in order to strengthen ties”
"""

In [126]:
# In the given article about Indian Prime Minister Narendra Modi's US visit, both positive and negative points can be identified. Let's break it down:

# **Positive Points**:

# 1. The US is seeking stronger ties with India, emphasizing the importance of India in US foreign relations.
# 2. The White House is hosting a state dinner in Modi’s honour, showcasing the growing relationship between the two countries.
# 3. The US and India have stepped up cooperation in areas such as trade and arms sales.
# 4. National Security Advisor Jake Sullivan’s trip to New Delhi underscored the dynamism of the US-India partnership.
# 5. The US sees India as a potential counterweight to China, highlighting India's strategic importance.
# 6. The US-India relationship is likely to grow with their shared concern over China.
# 7. Despite challenges, India has been skillful in maintaining relationships with both Russia and the US.
# 8. Modi’s trip has been welcomed with bipartisan support, emphasizing the political consensus around the importance of India-US ties.
# 9. Members of the US House of Representatives and Senate hailed the visit as a sign of the “enduring friendship” between the two countries.

# **Negative Points**:

# 1. Human rights groups criticize the state dinner as an endorsement of India’s far-right turn under Modi.
# 2. Allegations about India’s Muslims and other minorities experiencing increased violence and repression under Modi's government.
# 3. Modi has been criticized for consolidating power and cracking down on dissent.
# 4. The White House has been challenged for honoring a leader with controversial views.
# 5. Advocacy groups are demanding scrutiny of India’s human rights record.
# 6. Modi had previously been banned from entering the US due to allegations related to anti-Muslim violence in Gujarat in 2002.
# 7. Modi tried to ban a BBC documentary covering the 2002 riots in Gujarat.
# 8. Human rights groups accuse Modi's government of pursuing a far-right Hindu nationalist agenda.
# 9. The US State Department and the US Commission on International Religious Freedom have expressed concerns about religious freedom issues in India.
# 10. Politicians in states controlled by Modi's Bharatiya Janata Party (BJP) who promote violent anti-Muslim rhetoric face little repercussions.
# 11. Shidore mentions that Modi's human rights record has been overlooked, suggesting the US might be prioritizing strategic concerns over human rights.

# In total, there are **9 positive points** and **11 negative points** in the article. It's worth noting that the classification of points as positive or negative is based on the content and tone of the article, and different interpretations may result in different classifications.

In [127]:
num_pos_pts9 = 9
num_neg_pts9 =11

In [128]:
url10 = 'https://www.reuters.com/world/biden-modi-strengthen-ties-with-defense-trade-agreements-2023-06-22/' 


sentence10 = """
Biden, Modi hail new era for US-India ties and tout deals | Reuters
Modi hail new era for US-India ties and tout deals
on chips, minerals, tech, space and defenseHuman rights, democracy in focusModi addresses press,
CongressWASHINGTON, June 22 (Reuters) - U.S. President Joe Biden and Narendra Modi hailed a new era in their countries' relationship after the White House rolled out the red carpet for the Indian prime minister on Thursday, touting deals on defense and commerce aimed at countering China's global influence.“Two great nations, two great friends, and two great powers. Cheers," Biden told Modi in a toast at a state dinner. Modi said in reply: “You are soft spoken, but when it comes to action, you are very strong.”Though the countries are not formal treaty-bound allies and India has long relished its independence, Washington wants Delhi to be a strategic counterweight to China. While neither leader criticized Beijing directly in official statements, they alluded to the Xi Jinping-led government."The dark clouds of coercion and confrontation are casting their shadow in the Indo Pacific," Modi told Congress. "The stability of the region has become one of the central concerns of our partnership."After Biden and Modi spoke privately for more than two hours, a joint statement included a warning of rising tensions and destabilizing actions in the East and South China Sea and stressed the importance of international law and freedom of navigation."The challenges and opportunities facing the world in this century require that India and the United States work and lead together, and we are," Biden said as he welcomed Modi to the White House.The state dinner, held in a large, elaborately erected tent on the White House South Lawn, featured a number of Silicon Valley executives on the guest list, including Apple's Tim Cook, 23andMe CEO Anne Wojcicki, Google CEO Sundar Pichai, OpenAI CEO Sam Altman, among others.With the slightly Indian-accented dinner, the Bidens followed a White House tradition of serving guests from abroad food that has faint echoes of home with a strong American bent.Modi is seeking to raise the status that India, the world's most populous country at 1.4 billion and fifth-largest economy, has on the world stage as a manufacturing and diplomatic powerhouse while navigating strained ties with China.His participation in a White House press conference itself was a reflection of contrasting political traditions, marking the first time Modi has taken questions in such a format in his nine-year tenure. He took one question apiece from an American and an Indian journalist selected in advance, and dismissed criticism about India's human rights record.Washington has been frustrated by India's close ties with Russia while Moscow wages war in Ukraine. Modi avoided direct mention of Russia, but told lawmakers the conflict was "causing great pain in the region. Since it involves major powers, the outcomes are severe."BOYCOTTS AND WELCOME CROWDSModi's visit was not without controversy. The speech to Congress, normally a bipartisan affirmation of a visiting leader from an allied nation, was boycotted by some liberal lawmakers, who cited the Modi government's treatment of India's Muslim minority."I encourage my colleagues who stand for pluralism, tolerance and freedom of the press to join me in doing the same," Representative Alexandria Ocasio-Cortez tweeted on Wednesday.[1/12]U.S. President Joe Biden and India's Prime Minister Narendra Modi raise a toast during an official state dinner at the White House in Washington. REUTERS/Elizabeth FrantzModi's speech to the Congress also drew a crowd of on-lookers who chanted "Modi, Modi" at applause lines.As some 7,000 well-wishers gathered for a colorful opening ceremony at the White House, a far smaller group of demonstrators gathered blocks away to protest the Biden administration's coziness with Modi, who leads the Hindu nationalist Bharatiya Janata Party.Asked by a U.S. reporter about what steps he would take to improve the rights of Muslims and other minorities, Modi said "there's absolutely no space for discrimination" in India.Biden told reporters he and Modi had a straightforward discussion about democratic values.NEW US-INDIA DEALSThe two countries announced agreements on semiconductors, critical minerals, technology, space cooperation and defense cooperation and sales.Some are aimed at diversifying supply chains to reduce dependence on China. Others are aimed at cornering the market in advanced technologies that may feature on the battlefields of the future. They also ended disputes at the World Trade Organization, and India removed some tariffs on U.S. goods.The United States is India's largest trading partner but the U.S. has much larger trading relationships with China, the EU, and North American neighbors.Biden and Modi signed off on a deal to allow General Electric (GE.N) to produce jet engines in India to power Indian military aircraft, through an agreement with Hindustan Aeronautics. (HIAE.NS)U.S. Navy ships in the region will be able to stop in Indian shipyards for repairs under a maritime agreement, and India will procure U.S.-made armed MQ-9B SeaGuardian drones.U.S. chipmaker Micron Technology's (MU.O) plans a $2.7 billion semiconductor testing and packaging unit, to be built in Modi's home state of Gujarat. The U.S. will also make it easier for skilled Indian workers to get and renew U.S. visas.India also agreed to join the U.S.-led Artemis Accords on space exploration and to work with NASA on a joint mission to the International Space Station in 2024.Indian Foreign Secretary Vinay Kwatra said the visit so far had been "truly pathbreaking" and New Delhi is confident it will help take the relationship with Washington to "higher reaches"."The richness of form and substance of the visit clearly tells you that it’s an exceptional, landmark, pathbreaking visit," Kwatra told reporters."The decisions taken during the visit are truly transformative across a wide range of areas. Naturally, it is something which is possible when the countries have deep trust in each other and are in it for long term

"""

In [129]:
# Let's break down the content into positive and negative points regarding the relationship between the US and India, as well as comments related to Prime Minister Modi.

# **Positive Points**:
# 1. Biden and Modi hailed a new era in the US-India relationship.
# 2. Deals were touted on defense and commerce.
# 3. Both nations shared sentiments of being great powers and friends.
# 4. Emphasis on the strategic partnership, especially in relation to the Indo-Pacific and China.
# 5. The White House hosted a state dinner for PM Modi.
# 6. Presence of top Silicon Valley executives at the state dinner.
# 7. The dinner reflected a blend of Indian and American traditions.
# 8. Modi aims to raise India's global stature.
# 9. Biden and Modi had productive private conversations.
# 10. Enthusiastic crowd cheering for Modi at Congress.
# 11. New US-India deals spanning various sectors.
# 12. Agreements to diversify supply chains and reduce dependence on China.
# 13. Military cooperation agreements.
# 14. Semiconductor unit by Micron Technology in Gujarat.
# 15. Easier visa processes for skilled Indian workers.
# 16. Cooperation in space exploration with the Artemis Accords and joint mission to the International Space Station.
# 17. Vinay Kwatra’s comments on the visit being "truly pathbreaking".

# **Negative Points**:
# 1. Neither the US nor India are formal treaty-bound allies.
# 2. Modi's visit faced controversy.
# 3. Some liberal lawmakers boycotted Modi's speech to Congress due to concerns about treatment of India's Muslim minority.
# 4. Demonstrators protested the Biden administration's closeness with Modi.
# 5. AOC's tweet voicing concerns about Modi's administration.
# 6. Washington has been frustrated by India's close ties with Russia.
# 7. Modi's prior ban from the US due to allegations related to violence in Gujarat in 2002.
# 8. Questions raised about India's human rights record.
# 9. Criticism about Modi’s treatment of the Muslim minority and other issues.

# In summary, the article contains **17 positive points** and **9 negative points**.

In [130]:
num_pos_pts10 = 17
num_neg_pts10 = 9

In [131]:
url11 = 'https://www.theguardian.com/us-news/2023/jun/22/joe-biden-narendra-modi-visit-washington' 


sentence11 = """


Biden announces raft of new deals with Modi amid calls to address human rights concerns,A handful of lawmakers boycotted the Indian prime minister’s address to Congress, while protesters gathered near the White HouseMary Yang in Washington. Joe Biden has pronounced the US-India relationship never stronger and rolled out a series of new business deals on the second day of the Indian prime minister’s state visit, but Narendra Modi’s trip has been marked with controversy as many call for Biden to address India’s continuing crackdown on religious and press freedoms.An official state visit is the highest-ranked diplomatic invitation extended to a foreign leader and is a sign of the US president’s national security priorities as his administration courts India as a partner against China.‘India is now a linchpin’: US looks to Narendra Modi’s visit to counter ChinaRead moreIn a joint news conference with Modi, Biden called the relationship between the US and India among the most consequential in the world and “more dynamic than at any time in history.”“We talked about our shared efforts to mitigate humanitarian tragedies unleashed by Russia’s brutal war in Ukraine and to defend core principles of the UN charter,” the US president said. Modi has previously faced criticism for not more forcefully criticising Russia’s war in Ukraine.“Peace and security in the Indo-Pacific is our priority,” Modi said. “India and the US stand shoulder to shoulder in our fight against terrorism and fundamentalism.”When asked about the human rights and democratic issues in India, Biden said: “The prime minister and I had a good discussion about democratic values – that’s the nature of our relationship, we’re straightforward with each other.”Modi said he was surprised by the critique. “There is absolutely no space for discrimination,” he said. “In India the benefits provided by the government is accessible to all.”Among the announcements made on Thursday is an agreement that will allow US-based General Electric to partner with India-based Hindustan Aeronautics to produce jet engines for Indian aircraft in India and the sale of US-made armed MQ-9B SeaGuardian drones.The Biden administration also announced plans to bolster India’s semiconductor industry.“We made critical and emerging technologies the pillar of our next generation partnership to ensure these technologies promote and protect our values, remain open, accessible, trusted and secure,” Biden said. “All this matters for America, for India and for the world.”Thousands of Indian Americans and other guests gathered on the White House south lawn to greet Modi on Thursday morning, ahead of the private meeting between the two leaders and a state dinner. Modi, India’s prime minister since 2014, also addressed a joint session of Congress on Thursday afternoon.At the same time, dozens of demonstrators gathered near the White House gates to protest the visit. Human rights advocates have accused Modi and his party, the BJP, of fueling the rise of violence associated with Hindu nationalism and discriminating against India’s Muslim minority.“The fact that Modi has been invited despite being complicit with innocent people being killed since 2002 when he was the chief minister [of Gujarat] is devastating for our communities because it’s just saying, ‘it’s OK what you do back home but you can still come here,’” said Husnaa Vhora, the advocacy associate for the DC-based non-profit Indian American Muslim council.h the top stories from the US, plus the day’s must-reads from across the GuardianPrivacy Notice: Newsletters may contain info about charities, online ads, and content funded by outside parties. What is the BBC Modi documentary and why is it so controversial?Read moreDozens of lawmakers had also urged Biden to address the issue of human rights with Modi in a letter on Tuesday, asking the president to raise reports of “shrinking of political space, the rise of religious intolerance, the targeting of civil society organisations and journalists and growing restrictions on press freedoms and internet access” in India.A handful of representatives, including Rashida Tlaib and Ilhan Omar, who are Muslim, boycotted Modi’s address to Congress. The Indian leader focused on unity in that message.“The beauty of democracy is the constant connect with the people, to listen to them and feel their pulse,” said Modi, who shook hands with applauding members of Congress as he walked down to the front. It was his second time addressing both chambers of Congress, the first being in 2016. “We are home to all faiths in the world, and we celebrate all of them,” Modi claimed in his speech on Thursday.As expected, Biden balanced lawmakers and human rights advocates’ concerns with the US’s desire to deepen ties with India, a major trading partner and a counterweight to China.“China is in the background of all of these meetings,” said Tamanna Salikuddin, the director of South Asia programs at the non-partisan US Institute of Peace. “There is real convergence between the US and India on China.”Since 2020, India has seen deadly confrontations along its border with China, and security in the Indo-Pacific has become a shared priority. “India is, I think, one of the leading partners we have in [the Biden administration’s] Indo-Pacific strategy,” she said

"""

In [132]:
# From the provided content, let's identify the positive and negative points:

# **Positive Points**:
# 1. Biden pronounced the US-India relationship as never stronger.
# 2. Several new business deals were announced during Modi's visit.
# 3. An official state visit is a significant diplomatic gesture, reflecting the importance of the relationship.
# 4. Biden and Modi's joint news conference emphasized the consequential and dynamic nature of the US-India relationship.
# 5. Biden and Modi discussed efforts related to the situation in Ukraine.
# 6. Modi stressed peace, security in the Indo-Pacific, and the joint fight against terrorism.
# 7. New agreements were announced, such as the partnership between General Electric and Hindustan Aeronautics.
# 8. Biden's plan to strengthen India’s semiconductor industry.
# 9. Thousands of Indian Americans and other guests gathered in support of Modi at the White House.
# 10. Modi addressed a joint session of Congress and focused on the theme of unity.
# 11. Modi emphasized the diverse nature of India's democracy and its inclusivity of different faiths.
# 12. A shared understanding between the US and India concerning China, as mentioned by Tamanna Salikuddin.
# 13. India's role as a significant partner in the Biden administration’s Indo-Pacific strategy.

# **Negative Points**:
# 1. Modi's trip was marred by controversy concerning human rights and press freedom in India.
# 2. Criticism against Modi for not more forcefully criticizing Russia's war in Ukraine.
# 3. Concerns raised about human rights and democratic values in India.
# 4. Protests near the White House gates during Modi's visit.
# 5. Accusations against Modi and BJP for promoting violence linked to Hindu nationalism and discrimination against the Muslim minority.
# 6. Husnaa Vhora's comment on Modi being invited despite past controversies.
# 7. Lawmakers urged Biden to address human rights issues with Modi.
# 8. Reports cited concerns of shrinking political space, religious intolerance, and other issues in India.
# 9. A boycott of Modi’s address to Congress by a few representatives.
# 10. Concerns from human rights advocates regarding the US's balance of interests with India.
# 11. Background tension related to China influencing US-India meetings.

# In summary, the article presents **13 positive points** and **11 negative points**.

In [133]:
num_pos_pts11 = 13
num_neg_pts11 = 11

In [134]:
#############   Roberta Pretrained Model  

In [135]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [136]:
sentences = [sentence1, sentence2, sentence3, sentence4,sentence5, sentence6, sentence7, sentence8, sentence9,sentence10,sentence11]
url = [url1, url2, url3, url4, url5, url6, url7, url8, url9, url10, url11]
num_pos_pts = [num_pos_pts1, num_pos_pts2, num_pos_pts3, num_pos_pts4, num_pos_pts5, num_pos_pts6, num_pos_pts7, num_pos_pts8, num_pos_pts9, num_pos_pts10, num_pos_pts11]
num_neg_pts = [num_neg_pts1, num_neg_pts2, num_neg_pts3, num_neg_pts4, num_neg_pts5, num_neg_pts6, num_neg_pts7, num_neg_pts8, num_neg_pts9, num_neg_pts10, num_neg_pts11]


In [137]:
sentences = [str(sentence) for sentence in sentences]

In [138]:

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


News_url = []
negative= []
neutral= []
positive= []

num_pos_pts1= []
num_neg_pts1= []


for i in sentences:

    txt = i

    position = sentences.index(i)

    tokens = tokenizer.encode_plus(txt, add_special_tokens=False,
                                   return_tensors='pt')

    print(len(tokens['input_ids'][0]))
    tokens

    a = torch.arange(30)
    a
    torch.split(a, 4)

    input_id_chunks = tokens['input_ids'][0].split(510)
    mask_chunks = tokens['attention_mask'][0].split(510)


    a = torch.cat(
        [torch.Tensor([101]), a, torch.Tensor([102])]
    )

    a

    padding_len = 80 - a.shape[0]

    padding_len


    if padding_len > 0:
        a = torch.cat(
            [a, torch.Tensor([0] * padding_len)]
        )

    a


    # define target chunksize
    chunksize = 512

    # split into chunks of 510 tokens, we also convert to list (default is tuple which is immutable)
    input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
    mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))

    # loop through each chunk
    for i in range(len(input_id_chunks)):
        
        input_id_chunks[i] = torch.cat([
            torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
        ])
        # add attention tokens to attention mask
        mask_chunks[i] = torch.cat([
            torch.tensor([1]), mask_chunks[i], torch.tensor([1])
        ])
        # get required padding length
        pad_len = chunksize - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i], torch.Tensor([0] * pad_len)
            ])
            mask_chunks[i] = torch.cat([
                mask_chunks[i], torch.Tensor([0] * pad_len)
            ])

    # check length of each tensor
    for chunk in input_id_chunks:
        print(len(chunk))
  


    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)

    input_dict = {
        'input_ids': input_ids.long(),
        'attention_mask': attention_mask.int()
    }
    input_dict

    outputs = model(**input_dict)
    probs = torch.nn.functional.softmax(outputs[0], dim=-1)
    probs

    probs = probs.mean(dim=0)
    probs


    import re

    # Sample string representation of the tensor
    tensor_str = f"{probs}"

    # Extract the numbers using regex
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", tensor_str)

    # Convert the extracted numbers to float and store them in a list
    output_list = [float(num) for num in numbers]

    print(output_list[0:3])


    News_url.append(url[position])
    
    num_pos_pts1.append(num_pos_pts[position])
    num_neg_pts1.append(num_neg_pts[position])
    
    
    negative.append(output_list[0])
    neutral.append(output_list[1])
    positive.append(output_list[2])

    time.sleep(0.2)


data = {
    'News_article_url': News_url,
    'negative_score': negative,
    'neutral': neutral,
    'positive_score': positive,
    'num_pos_pts': num_pos_pts1,
    'num_neg_pts': num_neg_pts1,
    
    
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Print the final DataFrame
print(df)




4283
512
512
512
512
512
512
512
512
512
[0.2787, 0.5991, 0.1221]
1175
512
512
512
[0.0289, 0.4338, 0.5373]
1013
512
512
[0.0178, 0.3666, 0.6156]
732
512
512
[0.011, 0.4457, 0.5433]
1578
512
512
512
512
[0.0126, 0.671, 0.3164]
784
512
512
[0.0184, 0.6133, 0.3683]
1050
512
512
512
[0.0133, 0.3358, 0.6509]
2523
512
512
512
512
512
[0.1148, 0.6261, 0.2591]
1798
512
512
512
512
[0.2585, 0.6528, 0.0887]
1294
512
512
512
[0.0904, 0.5362, 0.3735]
1146
512
512
512
[0.1259, 0.6231, 0.251]
                                     News_article_url  negative_score  \
0   https://thewire.in/security/modi-us-visit-secu...          0.2787   
1   https://www.indiatoday.in/india/story/modis-tr...          0.0289   
2   https://indianexpress.com/article/india/narend...          0.0178   
3   https://www.thehindu.com/news/national/india-u...          0.0110   
4   https://www.firstpost.com/explainers/pm-narend...          0.0126   
5   https://www.ndtv.com/india-news/us-media-cover...          0.0184   
6   

In [139]:
full_article = df.copy()
full_article['sentence_number'] = full_article.index + 1
full_article

,News_article_url,negative_score,neutral,positive_score,num_pos_pts,num_neg_pts,sentence_number
0,https://thewire.in/security/modi-us-visit-secu...,0.2787,0.5991,0.1221,12,18,1
1,https://www.indiatoday.in/india/story/modis-tr...,0.0289,0.4338,0.5373,16,0,2
2,https://indianexpress.com/article/india/narend...,0.0178,0.3666,0.6156,14,0,3
3,https://www.thehindu.com/news/national/india-u...,0.0110,0.4457,0.5433,13,0,4
4,https://www.firstpost.com/explainers/pm-narend...,0.0126,0.6710,0.3164,27,0,5
5,https://www.ndtv.com/india-news/us-media-cover...,0.0184,0.6133,0.3683,12,2,6
6,https://www.financialexpress.com/business/defe...,0.0133,0.3358,0.6509,16,2,7
7,https://www.cnn.com/2023/06/21/politics/joe-bi...,0.1148,0.6261,0.2591,13,12,8
8,https://www.aljazeera.com/news/2023/6/21/whats...,0.2585,0.6528,0.0887,9,11,9
9,https://www.reuters.com/world/biden-modi-stren...,0.0904,0.5362,0.3735,17,9,10


In [140]:
full_article['rank_positive_score'] = full_article['positive_score'].rank(method='average', ascending=True)
full_article['rank_negative_score'] = full_article['negative_score'].rank(method='average', ascending=False)
full_article['rank_average'] = (full_article['rank_positive_score'] +full_article['rank_negative_score'] )/2
full_article['rank_average_normalized'] = (full_article['rank_average'] - full_article['rank_average'].min()) / (full_article['rank_average'].max() - full_article['rank_average'].min())
full_article['positive_score_negative_score_pts_count'] = (full_article['num_pos_pts']-full_article['num_neg_pts'])/(full_article['num_pos_pts']+full_article['num_neg_pts'])
full_article['positive_score_negative_score_pts_count'] = (full_article['num_pos_pts']-full_article['num_neg_pts'])/(full_article['num_pos_pts']+full_article['num_neg_pts'])
full_article['positive_score_negative_score_pts_count_normalized'] = (full_article['positive_score_negative_score_pts_count'] - full_article['positive_score_negative_score_pts_count'].min()) / (full_article['positive_score_negative_score_pts_count'].max() - full_article['positive_score_negative_score_pts_count'].min())
full_article = full_article.rename(columns={'rank_average_normalized': 'Tone_Score', 'positive_score_negative_score_pts_count_normalized': 'Topic_Bias_Score'})
full_article['Article_bias_score'] = full_article['Tone_Score']+full_article['Topic_Bias_Score']
full_article = full_article.sort_values(by=['Article_bias_score'], ascending=[ False])
full_article


,News_article_url,negative_score,neutral,positive_score,num_pos_pts,num_neg_pts,sentence_number,rank_positive_score,rank_negative_score,rank_average,Tone_Score,positive_score_negative_score_pts_count,Topic_Bias_Score,Article_bias_score
3,https://www.thehindu.com/news/national/india-u...,0.0110,0.4457,0.5433,13,0,4,9.0,11.0,10.0,1.000000,1.000000,1.000000,2.000000
2,https://indianexpress.com/article/india/narend...,0.0178,0.3666,0.6156,14,0,3,10.0,8.0,9.0,0.882353,1.000000,1.000000,1.882353
6,https://www.financialexpress.com/business/defe...,0.0133,0.3358,0.6509,16,2,7,11.0,9.0,10.0,1.000000,0.777778,0.814815,1.814815
4,https://www.firstpost.com/explainers/pm-narend...,0.0126,0.6710,0.3164,27,0,5,5.0,10.0,7.5,0.705882,1.000000,1.000000,1.705882
1,https://www.indiatoday.in/india/story/modis-tr...,0.0289,0.4338,0.5373,16,0,2,8.0,6.0,7.0,0.647059,1.000000,1.000000,1.647059
5,https://www.ndtv.com/india-news/us-media-cover...,0.0184,0.6133,0.3683,12,2,6,6.0,7.0,6.5,0.588235,0.714286,0.761905,1.350140
9,https://www.reuters.com/world/biden-modi-stren...,0.0904,0.5362,0.3735,17,9,10,7.0,5.0,6.0,0.529412,0.307692,0.423077,0.952489
7,https://www.cnn.com/2023/06/21/politics/joe-bi...,0.1148,0.6261,0.2591,13,12,8,4.0,4.0,4.0,0.294118,0.040000,0.200000,0.494118
10,https://www.theguardian.com/us-news/2023/jun/2...,0.1259,0.6231,0.2510,13,11,11,3.0,3.0,3.0,0.176471,0.083333,0.236111,0.412582
8,https://www.aljazeera.com/news/2023/6/21/whats...,0.2585,0.6528,0.0887,9,11,9,1.0,2.0,1.5,0.000000,-0.100000,0.083333,0.083333


In [141]:
## Final Bias order of the news articles of diffrent sources from favourable to unfavourable is as given below
full_article = full_article[['News_article_url','Topic_Bias_Score','Tone_Score','Article_bias_score']]
full_article

,News_article_url,Topic_Bias_Score,Tone_Score,Article_bias_score
3,https://www.thehindu.com/news/national/india-u...,1.000000,1.000000,2.000000
2,https://indianexpress.com/article/india/narend...,1.000000,0.882353,1.882353
6,https://www.financialexpress.com/business/defe...,0.814815,1.000000,1.814815
4,https://www.firstpost.com/explainers/pm-narend...,1.000000,0.705882,1.705882
1,https://www.indiatoday.in/india/story/modis-tr...,1.000000,0.647059,1.647059
5,https://www.ndtv.com/india-news/us-media-cover...,0.761905,0.588235,1.350140
9,https://www.reuters.com/world/biden-modi-stren...,0.423077,0.529412,0.952489
7,https://www.cnn.com/2023/06/21/politics/joe-bi...,0.200000,0.294118,0.494118
10,https://www.theguardian.com/us-news/2023/jun/2...,0.236111,0.176471,0.412582
8,https://www.aljazeera.com/news/2023/6/21/whats...,0.083333,0.000000,0.083333
